In [ ]:
import pandas as pd
import warnings
# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
print("🔄 Bắt đầu đọc dữ liệu Parquet...")
parquet_folder = r"C:\Khue\TDN\data\processed" # <-- Đảm bảo đường dẫn này chính xác

# Tự động tìm tất cả các file parquet theo mẫu
try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [os.path.join(parquet_folder, f)
                     for f in all_files_in_folder
                     if f.startswith("sanluong_") and f.endswith(".parquet")]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file nào khớp mẫu 'sanluong_*.parquet' trong thư mục: {parquet_folder}")
        # Có thể dừng ở đây hoặc xử lý tiếp nếu cần
        # exit() # Bỏ comment nếu muốn dừng hẳn script
        df_all = pd.DataFrame() # Tạo DataFrame rỗng để code không lỗi ở các bước sau
    else:
        # Đọc và ghép toàn bộ file
        df_list = []
        for f in parquet_files:
            try:
                df_list.append(pd.read_parquet(f, engine='pyarrow')) # engine='pyarrow' thường là mặc định và nhanh
            except Exception as e:
                print(f"❌ Lỗi khi đọc file {f}: {e}")
        
        if df_list:
             df_all = pd.concat(df_list, ignore_index=True)
             print(f"✅ Đã đọc xong {len(parquet_files)} file parquet.")
             print(f"👉 Tổng số dòng dữ liệu ban đầu: {df_all.shape[0]:,}")
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame() # Tạo DataFrame rỗng

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame() # Tạo DataFrame rỗng
except Exception as e:
    print(f"❌ Lỗi không xác định khi truy cập thư mục hoặc file: {e}")
    df_all = pd.DataFrame() # Tạo DataFrame rỗng

# ==================== CELL 2: CHỌN CỘT VÀ ĐỔI TÊN ====================
if not df_all.empty:
    # Chọn các cột cần thiết và TẠO BẢN SAO (.copy()) để tránh SettingWithCopyWarning
    required_cols = ['CTDL', 'NMTD', 'MADIEMDO', 'ENDTIME', 'CS']
    if all(col in df_all.columns for col in required_cols):
        df_sanluong = df_all[required_cols].copy()
        # Đổi tên cột ENDTIME thành TIME
        df_sanluong.rename(columns={'ENDTIME': 'TIME'}, inplace=True)
    else:
        print(f"❌ Thiếu cột cần thiết trong dữ liệu. Cần có: {required_cols}")
        missing = [col for col in required_cols if col not in df_all.columns]
        print(f"   -> Các cột bị thiếu: {missing}")
        df_sanluong = pd.DataFrame() # Tạo rỗng để tránh lỗi sau
else:
    print("ℹ️ Bỏ qua bước chuẩn bị dữ liệu sản lượng do không có dữ liệu đầu vào.")
    df_sanluong = pd.DataFrame()

# ==================== CELL 3: XỬ LÝ THỜI GIAN, TẠO UI VÀ VẼ BIỂU ĐỒ ====================

# Chỉ tiếp tục nếu df_sanluong có dữ liệu
if not df_sanluong.empty:
    print("🔄 Xử lý thời gian và tối ưu DataFrame...")
    # ========== TỐI ƯU HÓA: XỬ LÝ DATETIME VÀ ĐẶT INDEX ==========
    # 1. Đảm bảo cột TIME là datetime
    try:
        df_sanluong['TIME'] = pd.to_datetime(df_sanluong['TIME'])
    except Exception as e:
        print(f"❌ Lỗi chuyển đổi cột TIME sang datetime: {e}")
        # Cân nhắc dừng hoặc xử lý lỗi khác tại đây
        df_sanluong = pd.DataFrame() # Đặt lại thành rỗng nếu không xử lý được time

    if not df_sanluong.empty:
        # 2. Tạo các cột thời gian để index
        df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
        df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month
        df_sanluong['DAY_NUM'] = df_sanluong['TIME'].dt.day

        # 3. Đặt MultiIndex (KHÔNG bao gồm 'TIME' vì cần nó làm cột cho trục X)
        #    Index sẽ giúp tăng tốc lọc bằng .loc
        index_cols = ['CTDL', 'NMTD', 'YEAR', 'MONTH_NUM', 'DAY_NUM']
        # Kiểm tra sự tồn tại của các cột trước khi đặt index
        if all(col in df_sanluong.columns for col in index_cols):
             # Giữ lại các cột cần thiết cho plot ('TIME', 'CS', 'MADIEMDO') làm cột thường
            df_sanluong.set_index(index_cols, inplace=True)
            
            # 4. Sắp xếp Index - BƯỚC QUAN TRỌNG để .loc nhanh
            df_sanluong.sort_index(inplace=True)
            print("✅ Đã xử lý thời gian và tối ưu index DataFrame.")
        else:
            print("❌ Thiếu cột cần thiết để tạo index tối ưu.")
            missing_idx_cols = [col for col in index_cols if col not in df_sanluong.columns]
            print(f"   -> Các cột index bị thiếu: {missing_idx_cols}")
            # Nếu muốn vẫn chạy nhưng không tối ưu, có thể bỏ qua set_index và sort_index
            # Nếu không, đặt df_sanluong thành rỗng
            df_sanluong = pd.DataFrame()


# --- Các phần còn lại chỉ chạy nếu df_sanluong sau khi xử lý vẫn có dữ liệu ---
if not df_sanluong.empty:
    print("🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN ==========
    # Lấy từ index thay vì cột sau khi đã set_index
    try:
        ctdl_list = sorted(df_sanluong.index.get_level_values('CTDL').unique())
        year_list = sorted(df_sanluong.index.get_level_values('YEAR').unique())
    except Exception as e:
         print(f"❌ Lỗi khi lấy danh sách CTDL/Year từ index: {e}")
         ctdl_list = []
         year_list = []


    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width='250px', margin='0px 20px 0px 0px'):
        # Đảm bảo options không phải là None hoặc kiểu không phù hợp
        safe_options = list(options) if options is not None else []
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            layout=widgets.Layout(width=width, margin=margin),
            style={'description_width': 'auto'},
            disabled=not bool(safe_options) # Vô hiệu hóa nếu không có lựa chọn
        )

    # ========== DROPDOWNS ==========
    ctdl_dropdown_month = make_dropdown(ctdl_list, 'CTDL:', '300px')
    nmtd_dropdown_month = make_dropdown([], 'Nhà máy:', '300px') # Bắt đầu rỗng
    year_dropdown_month = make_dropdown(year_list, 'Năm:', '120px')
    month_dropdown_month = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
    day_placeholder = widgets.Label(value='', layout=widgets.Layout(width='120px', margin='0px 20px 0px 0px')) # Để căn chỉnh

    ctdl_dropdown_day = make_dropdown(ctdl_list, 'CTDL:', '300px')
    nmtd_dropdown_day = make_dropdown([], 'Nhà máy:', '300px') # Bắt đầu rỗng
    year_dropdown_day = make_dropdown(year_list, 'Năm:', '120px')
    month_dropdown_day = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
    day_dropdown_day = make_dropdown([], 'Ngày:', '120px') # Bắt đầu rỗng, sẽ được cập nhật

    # ========== NHÓM WIDGET ==========
    month_widgets = [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]
    day_widgets = [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]

    # ========== CẬP NHẬT NHÀ MÁY (Tối ưu bằng .loc trên index) ==========
    def update_nmtd(ctdl_value, dropdown):
        options = [] # Mặc định là rỗng
        if ctdl_value and not df_sanluong.empty: # Chỉ cập nhật nếu có giá trị CTDL và DataFrame không rỗng
            try:
                # Lấy danh sách NMTD duy nhất tại level 'NMTD' của index, ứng với ctdl_value đã chọn
                # df_sanluong.loc[ctdl_value] trả về DataFrame con với index cấp 1 là ctdl_value
                options = sorted(df_sanluong.loc[ctdl_value].index.get_level_values('NMTD').unique())
            except KeyError:
                # Trường hợp ctdl_value không tồn tại trong index (dù không nên xảy ra nếu list đúng)
                options = []
            except Exception as e:
                # Bắt các lỗi khác có thể xảy ra
                print(f" Lỗi khi cập nhật NMTD cho CTDL '{ctdl_value}': {e}")
                options = []
        
        # Cập nhật options và trạng thái disabled
        dropdown.options = options
        dropdown.disabled = not bool(options)
        # Nếu options cập nhật thành rỗng, reset giá trị về None
        if not options:
            dropdown.value = None


    # Gắn sự kiện observe và gọi lần đầu để khởi tạo NMTD
    # Sử dụng lambda để truyền đúng dropdown cần cập nhật
    ctdl_dropdown_month.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_month), names='value')
    ctdl_dropdown_day.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_day), names='value')

    # Khởi tạo NMTD ban đầu (nếu ctdl_dropdown có giá trị)
    if ctdl_dropdown_month.value:
         update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
    if ctdl_dropdown_day.value:
         update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)


    # ========== GIỚI HẠN NGÀY ==========
    def update_day_options(*args): # Chấp nhận các đối số dư thừa từ observe
        year = year_dropdown_day.value
        month = month_dropdown_day.value
        current_day_value = day_dropdown_day.value # Lưu giá trị ngày hiện tại

        if year and month:
            try:
                max_day = calendar.monthrange(year, month)[1]
                day_options = list(range(1, max_day + 1))
                day_dropdown_day.options = day_options
                day_dropdown_day.disabled = False
                # Cố gắng giữ lại giá trị ngày đã chọn nếu nó còn hợp lệ
                if current_day_value is not None and current_day_value in day_options:
                    day_dropdown_day.value = current_day_value
                else:
                    # Nếu không hợp lệ hoặc chưa có, đặt thành ngày 1
                     day_dropdown_day.value = 1 if day_options else None
            except Exception as e:
                # Xử lý lỗi nếu năm/tháng không hợp lệ (dù khó xảy ra với dropdown số)
                print(f" Lỗi khi lấy số ngày cho {month}/{year}: {e}")
                day_dropdown_day.options = []
                day_dropdown_day.disabled = True
                day_dropdown_day.value = None
        else:
            # Nếu chưa chọn Năm hoặc Tháng, vô hiệu hóa chọn Ngày
            day_dropdown_day.options = []
            day_dropdown_day.disabled = True
            day_dropdown_day.value = None

    # Gắn sự kiện observe và gọi lần đầu để khởi tạo Ngày
    year_dropdown_day.observe(update_day_options, names='value')
    month_dropdown_day.observe(update_day_options, names='value')
    update_day_options() # Gọi để khởi tạo lần đầu

    # ========== VÙNG OUTPUT CHO BIỂU ĐỒ ==========
    # Đặt trước để hàm plot_filtered có thể truy cập
    out = widgets.Output()

    # ========== VẼ BIỂU ĐỒ (Tối ưu bằng .loc trên index) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        # Xóa output cũ trước khi bắt đầu
        with out:
            clear_output(wait=True) # wait=True giảm nhấp nháy

            # Kiểm tra đầu vào cơ bản
            if not all([ctdl, nmtd, year, month]):
                print("Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return
            if mode == 'day' and not day:
                print("Vui lòng chọn Ngày để xem theo ngày.")
                return
            

            filtered_df = pd.DataFrame() # Khởi tạo DataFrame rỗng

            try:
                if mode == 'month':
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM)
                    # reset_index() để lấy lại các cột index và các cột dữ liệu thường
                    idx_query = (ctdl, nmtd, year, month)
                    if idx_query in df_sanluong.index: # Kiểm tra nhanh sự tồn tại của key chính
                         filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{month}/{year}"
                else: # mode == 'day'
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM, DAY_NUM)
                    idx_query = (ctdl, nmtd, year, month, day)
                    if idx_query in df_sanluong.index:
                        filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{day}/{month}/{year}"

            except KeyError:
                # Xử lý trường hợp tổ hợp index không tồn tại (dù đã kiểm tra sơ bộ)
                print(f"ℹ️ Không tìm thấy dữ liệu chính xác cho lựa chọn (KeyError).")
                filtered_df = pd.DataFrame() # Đảm bảo là DataFrame rỗng
            except Exception as e:
                print(f"❌ Lỗi khi lọc dữ liệu bằng .loc: {e}")
                filtered_df = pd.DataFrame() # Đảm bảo là DataFrame rỗng

            # Kiểm tra lại lần nữa sau khi lọc và reset_index
            if filtered_df.empty:
                print("ℹ️ Không có dữ liệu cho lựa chọn này.")
                return # Thoát hàm nếu không có dữ liệu

            # --- Vẽ biểu đồ (Giữ nguyên cấu hình) ---
            try:
                # Sắp xếp lại theo TIME sau khi reset_index (quan trọng cho biểu đồ đường)
                filtered_df = filtered_df.sort_values('TIME')

                fig = px.line(
                    filtered_df,
                    x='TIME',
                    y='CS',
                    color='MADIEMDO', # Giữ nguyên color
                    render_mode='webgl', # Giữ nguyên render_mode
                    markers=True if mode == 'day' else False # Giữ nguyên logic markers
                )

                fig.update_layout(
                    title=None, # Giữ nguyên
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref='paper', yref='paper',
                            x=0.45, y=1.1,
                            xanchor='center', yanchor='top',
                            showarrow=False,
                            font=dict(size=18, color='black')
                        )
                    ],
                    template='plotly_white', # Giữ nguyên
                    height=600, # Giữ nguyên
                    hovermode='x unified', # Giữ nguyên

                    xaxis_title=None, # Giữ nguyên
                    yaxis_title='Giá trị công suất', # Giữ nguyên

                    xaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        title_font=dict(size=14, color='black'), # Bỏ bold vì đặt lại bằng annotation
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),
                    yaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        rangemode='tozero',
                        title_font=dict(size=14, color='black', weight='bold'),
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),

                    margin=dict(l=60, r=40, t=80, b=80), # Giữ nguyên
                    paper_bgcolor='white', # Giữ nguyên
                    plot_bgcolor='#f0f8ff' # Giữ nguyên
                )

                # ✅ Tên trục X bằng annotation (Giữ nguyên)
                fig.add_annotation(
                    render_mode='webgl', # Giữ nguyên render_mode
                    markers=True if mode == 'day' else False # Giữ nguyên logic markers
                )

                fig.update_layout(
                    title=None, # Giữ nguyên
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref='paper', yref='paper',
                            x=0.45, y=1.1,
                            xanchor='center', yanchor='top',
                            showarrow=False,
                            font=dict(size=18, color='black')
                        )
                    ],
                    template='plotly_white', # Giữ nguyên
                    height=600, # Giữ nguyên
                    hovermode='x unified', # Giữ nguyên

                    xaxis_title=None, # Giữ nguyên
                    yaxis_title='Giá trị công suất', # Giữ nguyên

                    xaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        title_font=dict(size=14, color='black'), # Bỏ bold vì đặt lại bằng annotation
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),
                    yaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        rangemode='tozero',
                        title_font=dict(size=14, color='black', weight='bold'),
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),

                    margin=dict(l=60, r=40, t=80, b=80), # Giữ nguyên
                    paper_bgcolor='white', # Giữ nguyên
                    plot_bgcolor='#f0f8ff' # Giữ nguyên
                )

                # ✅ Tên trục X bằng annotation (Giữ nguyên)
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref='paper', yref='paper',
                    x=0.45, y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color='black')
                )

                fig.update_traces( # Giữ nguyên
                    line=dict(width=2),
                    hovertemplate=': %{y}'
                )
                
                fig.show() # Hiển thị biểu đồ trong Output 'out'

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ Plotly: {e}")
                # Có thể hiển thị lỗi chi tiết hơn nếu cần debug

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN ==========
    def on_change_month(change):
        # Kiểm tra xem các widget cần thiết có giá trị không
        if all(w.value is not None for w in [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]):
             # Chỉ gọi plot nếu là sự thay đổi có ý nghĩa (không phải reset về None)
            if change is None or change['new'] != change['old']:
                 plot_filtered(
                    'month',
                    ctdl_dropdown_month.value,
                    nmtd_dropdown_month.value,
                    year_dropdown_month.value,
                    month_dropdown_month.value
                 )
        # else: # Debugging (optional)
             # print("Debug Month: Một hoặc nhiều widget tháng chưa có giá trị.")


    def on_change_day(change):
        if all(w.value is not None for w in [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]):
            if change is None or change['new'] != change['old']:
                plot_filtered(
                    'day',
                    ctdl_dropdown_day.value,
                    nmtd_dropdown_day.value,
                    year_dropdown_day.value,
                    month_dropdown_day.value,
                    day_dropdown_day.value
                )
        # else: # Debugging (optional)
             # print("Debug Day: Một hoặc nhiều widget ngày chưa có giá trị.")


    # ========== HÀM TẮT / BẬT OBSERVER ==========
    # Giữ nguyên các hàm này, chúng quan trọng cho logic chuyển tab
    _observed_callbacks_month = {}
    _observed_callbacks_day = {}

    def disable_observe(widgets, callback_map, callback_func):
        for w in widgets:
            try:
                # Lưu trữ hàm callback để có thể gắn lại đúng hàm
                callback_map[w] = callback_func
                w.unobserve(callback_func, names='value')
                # print(f"Unobserved {w.description}") # Debug
            except ValueError:
                # print(f"Callback already removed for {w.description}") # Debug
                pass # Bỏ qua nếu callback đã được gỡ bỏ trước đó

    def enable_observe(widgets, callback_map):
        for w in widgets:
            if w in callback_map:
                try:
                    # Chỉ observe nếu widget không bị disabled
                    if not w.disabled:
                        w.observe(callback_map[w], names='value')
                        # print(f"Observed {w.description}") # Debug
                    # else:
                        # print(f"Skipped observe on disabled {w.description}") # Debug

                except Exception as e:
                    print(f"Error observing {w.description}: {e}") # Debug

    # ========== GẮN OBSERVER BAN ĐẦU ==========
    # Thay vì lặp, gọi enable_observe để gắn kèm kiểm tra disabled
    # Tạo map trước khi gọi enable_observe
    for w in month_widgets: _observed_callbacks_month[w] = on_change_month
    for w in day_widgets: _observed_callbacks_day[w] = on_change_day

    enable_observe(month_widgets, _observed_callbacks_month)
    enable_observe(day_widgets, _observed_callbacks_day)

    # ========== TẠO GIAO DIỆN ==========
    tab_thang = widgets.HBox([
        ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month, day_placeholder
    ], layout=widgets.Layout(justify_content='flex-start', align_items='flex-end', overflow='visible')) # Căn chỉnh đáy widget

    tab_ngay = widgets.HBox([
        ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day
    ], layout=widgets.Layout(justify_content='flex-start', align_items='flex-end', overflow='visible')) # Căn chỉnh đáy widget


    tabs = widgets.Tab(children=[tab_thang, tab_ngay])
    tabs.set_title(0, '📅 Xem theo tháng')
    tabs.set_title(1, '🗓️ Xem theo ngày')

    # ========== XỬ LÝ CHUYỂN TAB ==========
    def on_tab_change(change):
        if change['name'] == 'selected_index':
            new_tab_index = change['new']
            print(f"\n🔄 Chuyển sang tab: {tabs.get_title(new_tab_index)}")

            if new_tab_index == 1: # Chuyển sang tab NGÀY
                print("   Đồng bộ hóa lựa chọn từ Tháng sang Ngày...")
                # 1. Tắt observer của tab Ngày để tránh kích hoạt khi cập nhật giá trị
                disable_observe(day_widgets, _observed_callbacks_day, on_change_day)

                # 2. Đồng bộ giá trị từ tab Tháng sang tab Ngày
                ctdl_dropdown_day.value = ctdl_dropdown_month.value
                # update_nmtd sẽ được trigger bởi observe của ctdl_day khi bật lại,
                # nhưng ta cần gọi update trực tiếp để lấy đúng NMTD list trước
                update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)
                # Chỉ gán NMTD nếu giá trị tháng tồn tại trong options mới
                if nmtd_dropdown_month.value in nmtd_dropdown_day.options:
                     nmtd_dropdown_day.value = nmtd_dropdown_month.value
                else:
                     nmtd_dropdown_day.value = nmtd_dropdown_day.options[0] if nmtd_dropdown_day.options else None # Chọn cái đầu tiên hoặc None

                year_dropdown_day.value = year_dropdown_month.value
                month_dropdown_day.value = month_dropdown_month.value
                # update_day_options sẽ tự động được gọi khi month/year thay đổi (nếu observe đang bật)
                # Gọi lại để chắc chắn ngày được cập nhật đúng
                update_day_options()
                # Giá trị ngày sẽ được đặt về 1 (hoặc giá trị hợp lệ đầu tiên) bởi update_day_options

                # 3. Bật lại observer cho tab Ngày
                enable_observe(day_widgets, _observed_callbacks_day)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Ngày với giá trị mới
                print("   Vẽ biểu đồ cho tab Ngày...")
                on_change_day(None) # None để chỉ ra đây là gọi thủ công

            elif new_tab_index == 0: # Chuyển sang tab THÁNG
                print("   Đồng bộ hóa lựa chọn từ Ngày sang Tháng...")
                # 1. Tắt observer của tab Tháng
                disable_observe(month_widgets, _observed_callbacks_month, on_change_month)

                # 2. Đồng bộ giá trị từ tab Ngày sang tab Tháng
                ctdl_dropdown_month.value = ctdl_dropdown_day.value
                update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
                if nmtd_dropdown_day.value in nmtd_dropdown_month.options:
                    nmtd_dropdown_month.value = nmtd_dropdown_day.value
                else:
                    nmtd_dropdown_month.value = nmtd_dropdown_month.options[0] if nmtd_dropdown_month.options else None

                year_dropdown_month.value = year_dropdown_day.value
                month_dropdown_month.value = month_dropdown_day.value

                # 3. Bật lại observer cho tab Tháng
                enable_observe(month_widgets, _observed_callbacks_month)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Tháng
                print("   Vẽ biểu đồ cho tab Tháng...")
                on_change_month(None)

    tabs.observe(on_tab_change, names='selected_index')

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs, out)

    # ========== VẼ BIỂU ĐỒ BAN ĐẦU ==========
    # Gọi on_change_month để vẽ biểu đồ lần đầu tiên khi cell chạy xong (cho tab tháng mặc định)
    on_change_month(None)

elif df_all.empty:
     print("\n❌ Không có dữ liệu đầu vào từ file Parquet để xử lý và vẽ biểu đồ.")
else:
     print("\n❌ Đã xảy ra lỗi trong quá trình chuẩn bị dữ liệu. Không thể tạo giao diện.")

🔄 Bắt đầu đọc dữ liệu Parquet...
✅ Đã đọc xong 4 file parquet.
👉 Tổng số dòng dữ liệu ban đầu: 18,404,864
🔄 Xử lý thời gian và tối ưu DataFrame...


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import calendar
import numpy as np
import time  # Thêm để đo thời gian (tùy chọn)

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
print("🔄 Bắt đầu đọc dữ liệu Parquet...")
start_read_time = time.time()  # Đo thời gian đọc
parquet_folder = r"C:\Khue\TDN\data\processed"

try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [
        os.path.join(parquet_folder, f)
        for f in all_files_in_folder
        if f.startswith("sanluong_") and f.endswith(".parquet")
    ]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
        df_all = pd.DataFrame()
    else:
        df_list = []
        print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
        for i, f in enumerate(parquet_files):
            try:
                df_temp = pd.read_parquet(f, engine="pyarrow")
                # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                if all(col in df_temp.columns for col in required_cols_read):
                    df_list.append(df_temp[required_cols_read])  # Chỉ lấy cột cần thiết
                else:
                    print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
            except Exception as e:
                print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

        if df_list:
            print("   Ghép các DataFrame...")
            df_all = pd.concat(df_list, ignore_index=True)
            print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
            print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
            print(f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây")
            # Hiển thị thông tin bộ nhớ (tùy chọn)
            # df_all.info(memory_usage='deep')
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame()

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame()
except Exception as e:
    print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
    df_all = pd.DataFrame()


# ==================== CELL 2: TIỀN XỬ LÝ VÀ TỐI ƯU HÓA ====================
df_sanluong = pd.DataFrame()
df_sanluong_indexed = pd.DataFrame()

if not df_all.empty:
    print("\n🔄 Bắt đầu tiền xử lý và tối ưu hóa...")
    start_process_time = time.time()
    try:
        # 1. Đổi tên cột và đảm bảo kiểu dữ liệu datetime
        df_sanluong = df_all.rename(
            columns={"ENDTIME": "TIME"}, errors="raise"
        )  # Đảm bảo cột tồn tại
        df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])

        # 2. Đảm bảo kiểu dữ liệu số cho CS và xử lý NaN
        df_sanluong["CS"] = pd.to_numeric(df_sanluong["CS"], errors="coerce")
        initial_rows = len(df_sanluong)
        df_sanluong.dropna(subset=["CS"], inplace=True)
        removed_rows = initial_rows - len(df_sanluong)
        if removed_rows > 0:
            print(f"   ⚠️ Đã loại bỏ {removed_rows:,} dòng có giá trị CS không hợp lệ.")

        # 3. Tạo các cột thời gian cần thiết
        df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year.astype(
            "int16"
        )  # Tối ưu kiểu int
        df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month.astype("int8")
        df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day.astype("int8")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME"].dt.strftime("%H:%M")

        # 4. Tối ưu kiểu dữ liệu Categorical cho các cột chuỗi lặp lại
        df_sanluong["CTDL"] = df_sanluong["CTDL"].astype("category")
        df_sanluong["NMTD"] = df_sanluong["NMTD"].astype("category")
        df_sanluong["MADIEMDO"] = df_sanluong["MADIEMDO"].astype("category")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME_SLOT"].astype("category")

        # df_sanluong bây giờ chứa tất cả dữ liệu cần thiết với kiểu tối ưu
        print("   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.")
        # df_sanluong.info(memory_usage='deep') # Kiểm tra lại bộ nhớ

        # 5. Tạo DataFrame được Index cho tính toán thống kê nhanh
        index_cols = ["CTDL", "NMTD", "YEAR", "MONTH_NUM", "DAY_NUM"]
        # Chỉ cần TIME_SLOT và CS cho df_indexed
        df_sanluong_indexed = df_sanluong[index_cols + ["TIME_SLOT", "CS"]].copy()
        # Giữ kiểu category cho index để tiết kiệm bộ nhớ index
        df_sanluong_indexed.set_index(index_cols, inplace=True)
        df_sanluong_indexed.sort_index(inplace=True)  # Rất quan trọng cho hiệu năng loc
        print("   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.")
        # df_sanluong_indexed.info(memory_usage='deep')

        print(f"⏱️ Thời gian tiền xử lý: {time.time() - start_process_time:.2f} giây")

    except Exception as e:
        print(f"❌ Lỗi trong quá trình tiền xử lý: {e}")
        # Đảm bảo cả hai df đều rỗng nếu có lỗi
        df_sanluong = pd.DataFrame()
        df_sanluong_indexed = pd.DataFrame()
else:
    print("ℹ️ Không có dữ liệu đầu vào để xử lý.")

# ==================== CELL 3: GIAO DIỆN TƯƠNG TÁC VÀ VẼ BIỂU ĐỒ ====================

# --- Chỉ tiếp tục nếu dữ liệu đã được xử lý thành công ---
if not df_sanluong.empty and not df_sanluong_indexed.empty:
    print("\n🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN ==========
    try:
        ctdl_list = sorted(df_sanluong["CTDL"].cat.categories)
        year_list = sorted(df_sanluong["YEAR"].unique())
        nmtd_list_all = sorted(df_sanluong["NMTD"].cat.categories)
    except Exception as e:
        print(f"❌ Lỗi lấy danh sách dropdown: {e}")
        ctdl_list, year_list, nmtd_list_all = [], [], []

    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
        safe_options = list(options) if options is not None else []
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            value=safe_options[0] if safe_options else None,
            layout=widgets.Layout(width=width, margin=margin),
            style={"description_width": "auto"},
            disabled=not bool(safe_options),
        )

    # ========== DROPDOWNS ==========
    ctdl_dropdown_month_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_month_plot = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_month_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_month_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    placeholder_month_plot = widgets.Label(
        value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
    )
    ctdl_dropdown_day_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_day_plot = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_day_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_day_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    day_dropdown_day_plot = make_dropdown([], "Ngày:", "120px")
    ctdl_dropdown_stat = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_stat = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_stat = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_stat = make_dropdown(list(range(1, 13)), "Tháng:", "120px")

    # ========== NHÓM WIDGET ==========
    month_plot_widgets = [
        ctdl_dropdown_month_plot,
        nmtd_dropdown_month_plot,
        year_dropdown_month_plot,
        month_dropdown_month_plot,
    ]
    day_plot_widgets = [
        ctdl_dropdown_day_plot,
        nmtd_dropdown_day_plot,
        year_dropdown_day_plot,
        month_dropdown_day_plot,
        day_dropdown_day_plot,
    ]
    stat_widgets = [
        ctdl_dropdown_stat,
        nmtd_dropdown_stat,
        year_dropdown_stat,
        month_dropdown_stat,
    ]

    # ========== CẬP NHẬT NHÀ MÁY ==========
    nmtd_cache = {}

    def update_nmtd(ctdl_value, dropdown_to_update):
        # global nmtd_cache # <-- XÓA DÒNG NÀY
        options = []
        current_nmtd_value = dropdown_to_update.value
        if ctdl_value:
            if ctdl_value in nmtd_cache:
                options = nmtd_cache[ctdl_value]
            elif not df_sanluong_indexed.empty:
                try:
                    nmtd_options = (
                        df_sanluong_indexed.loc[ctdl_value]
                        .index.get_level_values("NMTD")
                        .unique()
                        .tolist()
                    )
                    options = sorted(nmtd_options)
                    nmtd_cache[ctdl_value] = options
                except KeyError:
                    options = []
                except Exception as e:
                    print(f"Lỗi NMTD cache cho '{ctdl_value}': {e}")
                    options = []
        dropdown_to_update.options = options
        dropdown_to_update.disabled = not bool(options)
        if current_nmtd_value in options:
            dropdown_to_update.value = current_nmtd_value
        elif options:
            dropdown_to_update.value = options[0]
        else:
            dropdown_to_update.value = None

    ctdl_dropdown_month_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_month_plot),
        names="value",
    )
    ctdl_dropdown_day_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_day_plot), names="value"
    )
    ctdl_dropdown_stat.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_stat), names="value"
    )
    if ctdl_dropdown_month_plot.value:
        update_nmtd(ctdl_dropdown_month_plot.value, nmtd_dropdown_month_plot)
    if ctdl_dropdown_day_plot.value:
        update_nmtd(ctdl_dropdown_day_plot.value, nmtd_dropdown_day_plot)
    if ctdl_dropdown_stat.value:
        update_nmtd(ctdl_dropdown_stat.value, nmtd_dropdown_stat)

    # ========== GIỚI HẠN NGÀY ==========
    def update_day_options(*args):
        year = year_dropdown_day_plot.value
        month = month_dropdown_day_plot.value
        current_day = day_dropdown_day_plot.value
        day_options = []
        if year and month:
            try:
                max_day = calendar.monthrange(year, month)[1]
                day_options = list(range(1, max_day + 1))
            except Exception as e:
                print(f"Lỗi lấy số ngày {month}/{year}: {e}")
        day_dropdown_day_plot.options = day_options
        day_dropdown_day_plot.disabled = not bool(day_options)
        if current_day in day_options:
            day_dropdown_day_plot.value = current_day
        elif day_options:
            day_dropdown_day_plot.value = 1
        else:
            day_dropdown_day_plot.value = None

    year_dropdown_day_plot.observe(update_day_options, names="value")
    month_dropdown_day_plot.observe(update_day_options, names="value")
    if year_dropdown_day_plot.value and month_dropdown_day_plot.value:
        update_day_options()

    # ========== VÙNG OUTPUT CHUNG ==========
    out = widgets.Output()

    # ========== VẼ BIỂU ĐỒ THEO MADIEMDO (Tab 1 & 2) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        # (Code hàm plot_filtered giữ nguyên)
        with out:
            clear_output(wait=True)
            print(f"--- BẮT ĐẦU VẼ: plot_filtered ({mode}) ---")
            plot_start_time = time.time()
            required_vals = [ctdl, nmtd, year, month]
            if mode == "day":
                required_vals.append(day)
            if not all(v is not None for v in required_vals):
                print("   ⚠️ Vui lòng chọn đủ các bộ lọc.")
                return
            filtered_df = pd.DataFrame()
            try:
                mask = (
                    (df_sanluong["CTDL"] == ctdl)
                    & (df_sanluong["NMTD"] == nmtd)
                    & (df_sanluong["YEAR"] == year)
                    & (df_sanluong["MONTH_NUM"] == month)
                )
                if mode == "day":
                    mask &= df_sanluong["DAY_NUM"] == day
                filtered_df = df_sanluong.loc[mask, ["TIME", "CS", "MADIEMDO"]].copy()
                filtered_df.sort_values("TIME", inplace=True)
                filter_time = time.time()
            except Exception as e:
                print(f"❌ Lỗi lọc: {e}")
                return
            if filtered_df.empty:
                print("   ℹ️ Không có dữ liệu.")
                return
            clear_output(wait=True)
            print(f"--- BẮT ĐẦU VẼ: plot_filtered ({mode}) ---")
            print(
                f"   ⏱️ Lọc: {filter_time - plot_start_time:.3f}s ({len(filtered_df)} dòng)"
            )
            try:
                title_ext = (
                    f"{month}/{year}" if mode == "month" else f"{day}/{month}/{year}"
                )
                fig = px.line(
                    filtered_df,
                    x="TIME",
                    y="CS",
                    color="MADIEMDO",
                    render_mode="webgl",
                    markers=True if mode == "day" else False,
                )
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.update_traces(
                    line=dict(width=2),
                    hovertemplate="Thời gian: %{x}<br>Công suất: %{y}",
                )
                draw_start_time = time.time()
                fig.show()
                print(f"   ⏱️ Vẽ PX xong: {time.time() - draw_start_time:.3f}s")
                print(f"   ✅ Hoàn thành {mode}.")
            except Exception as e:
                print(f"❌ Lỗi vẽ PX: {e}")

    # ========== VẼ BIỂU ĐỒ THỐNG KÊ THEO KHUNG GIỜ (Tab 3) ==========
    def plot_timeslot_stats(ctdl, nmtd, year, month):
        # (Code hàm plot_timeslot_stats giữ nguyên)
        with out:
            clear_output(wait=True)
            print(f"--- BẮT ĐẦU VẼ: plot_timeslot_stats ---")
            stat_start_time = time.time()
            if not all([ctdl, nmtd, year, month]):
                print("   ⚠️ Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return
            stats_by_time = pd.DataFrame()
            monthly_data_count = 0
            try:
                idx_query = (ctdl, nmtd, year, month)
                if idx_query in df_sanluong_indexed.index.droplevel("DAY_NUM").unique():
                    monthly_data = df_sanluong_indexed.loc[
                        idx_query, ["TIME_SLOT", "CS"]
                    ]
                    monthly_data_count = len(monthly_data)
                    filter_time = time.time()
                    if not monthly_data.empty:
                        stats_by_time = (
                            monthly_data.groupby("TIME_SLOT", observed=True)["CS"]
                            .agg(
                                min="min",
                                p25=lambda x: x.quantile(0.25),
                                p50="median",
                                p75=lambda x: x.quantile(0.75),
                                max="max",
                            )
                            .sort_index()
                        )
                        agg_time = time.time()
                    else:
                        print("   ℹ️ Không có dữ liệu chi tiết sau khi lọc.")

            except KeyError:
                print("   ℹ️ Không tìm thấy dữ liệu (KeyError).")
                return
            except Exception as e:
                print(f"❌ Lỗi tính toán thống kê: {e}")
                return
            if stats_by_time.empty:
                print("Không có dữ liệu thống kê ")
                return
            clear_output(wait=True)
            print(f"--- BẮT ĐẦU VẼ: plot_timeslot_stats ---")
            print(
                f"   ⏱️ Lọc index: {filter_time - stat_start_time:.3f}s ({monthly_data_count} dòng)"
            )
            if "agg_time" in locals():
                print(f"   ⏱️ Tính toán: {agg_time - filter_time:.3f}s")
            try:
                fig = go.Figure()
                time_slots = stats_by_time.index
                p25 = stats_by_time["p25"]
                p75 = stats_by_time["p75"]
                p50 = stats_by_time["p50"]
                min_cs = stats_by_time["min"]
                max_cs = stats_by_time["max"]
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=max_cs,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=min_cs,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(211, 211, 211, 0.5)",
                        name="Min-Max",
                        hoverinfo="skip",
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p75,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p25,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(173, 216, 230, 0.6)",
                        name="Tần suất 25-75",
                        hoverinfo="skip",
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p50,
                        mode="lines",
                        line=dict(color="blue", width=2.5),
                        name="Tần suất 50",
                        hovertemplate="Khung giờ: %{x}<br>Median: %{y:.2f}<extra></extra>",
                    )
                )
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Thống kê công suất theo khung giờ - {nmtd} - {month}/{year}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        tickmode="array",
                        tickvals=time_slots[::1],
                        ticktext=[t for t in time_slots],
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                        tickangle=-90,
                    ),
                    yaxis=dict(
                        rangemode="tozero",
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="right",
                        x=0.99,
                        bgcolor="rgba(255,255,255,0.7)",
                    ),
                    margin=dict(l=60, r=40, t=80, b=100),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Khung giờ</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.20,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                draw_start_time = time.time()
                fig.show()
                print(f"   ⏱️ Vẽ GO xong: {time.time() - draw_start_time:.3f}s")
                print("   ✅ Hoàn thành vẽ biểu đồ thống kê khung giờ.")
            except Exception as e:
                print(f"❌ Lỗi vẽ biểu đồ thống kê Plotly GO: {e}")

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN ==========
    def on_change_month_plot(change):
        if tabs.selected_index == 0:
            if all(w.value is not None for w in month_plot_widgets):
                if change is None or (
                    isinstance(change, dict) and change.get("new") != change.get("old")
                ):
                    plot_filtered(
                        "month",
                        ctdl_dropdown_month_plot.value,
                        nmtd_dropdown_month_plot.value,
                        year_dropdown_month_plot.value,
                        month_dropdown_month_plot.value,
                    )

    def on_change_day_plot(change):
        if tabs.selected_index == 1:
            if all(w.value is not None for w in day_plot_widgets):
                if change is None or (
                    isinstance(change, dict) and change.get("new") != change.get("old")
                ):
                    plot_filtered(
                        "day",
                        ctdl_dropdown_day_plot.value,
                        nmtd_dropdown_day_plot.value,
                        year_dropdown_day_plot.value,
                        month_dropdown_day_plot.value,
                        day_dropdown_day_plot.value,
                    )

    def on_change_stat(change):
        if tabs.selected_index == 2:
            if all(w.value is not None for w in stat_widgets):
                if change is None or (
                    isinstance(change, dict) and change.get("new") != change.get("old")
                ):
                    plot_timeslot_stats(
                        ctdl_dropdown_stat.value,
                        nmtd_dropdown_stat.value,
                        year_dropdown_stat.value,
                        month_dropdown_stat.value,
                    )

    # ========== HÀM TẮT / BẬT OBSERVER ==========
    _observed_callbacks = {}

    def disable_observe(widgets_list):
        for w in widgets_list:
            if w in _observed_callbacks and _observed_callbacks[w]["active"]:
                try:
                    w.unobserve(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = False
                except:
                    _observed_callbacks[w]["active"] = False

    def enable_observe(widgets_list):
        for w in widgets_list:
            if (
                w in _observed_callbacks
                and not w.disabled
                and not _observed_callbacks[w]["active"]
            ):
                try:
                    w.observe(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = True
                except Exception as e:
                    print(f"Lỗi observe {w.description}: {e}")

    # ========== GẮN OBSERVER BAN ĐẦU ==========
    for w in month_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_month_plot, "active": False}
    for w in day_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_day_plot, "active": False}
    for w in stat_widgets:
        _observed_callbacks[w] = {"func": on_change_stat, "active": False}

    # ========== TẠO GIAO DIỆN TABS ==========
    tab_month_plot = widgets.HBox(
        month_plot_widgets + [placeholder_month_plot],
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_day_plot = widgets.HBox(
        day_plot_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_stat = widgets.HBox(
        stat_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tabs = widgets.Tab(children=[tab_month_plot, tab_day_plot, tab_stat])
    tabs.set_title(0, "📅 Biểu đồ tháng")
    tabs.set_title(1, "🗓️ Biểu đồ ngày")
    tabs.set_title(2, "📊 Thống kê tháng")

    # ========== XỬ LÝ CHUYỂN TAB ==========
    # Sử dụng list để lưu trữ giá trị có thể thay đổi từ hàm bên trong
    current_tab_index_holder = [tabs.selected_index]  # Định nghĩa ở scope này

    def on_tab_change(change):
        if change["name"] == "selected_index":
            new_tab_index = change["new"]
            old_tab_index = current_tab_index_holder[0]
            if old_tab_index == new_tab_index:
                return

            print(
                f"\n🔄 Chuyển từ tab {old_tab_index} -> {new_tab_index} ({tabs.get_title(new_tab_index)})"
            )
            # --- Tắt observer tab cũ ---
            if old_tab_index == 0:
                disable_observe(month_plot_widgets)
            elif old_tab_index == 1:
                disable_observe(day_plot_widgets)
            elif old_tab_index == 2:
                disable_observe(stat_widgets)
            # --- Xác định nguồn đồng bộ ---
            source_widgets = {}
            if old_tab_index == 0:
                source_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }
            elif old_tab_index == 1:
                source_widgets = {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                }
            elif old_tab_index == 2:
                source_widgets = {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                }
            # --- Hàm phụ trợ đồng bộ ---
            print("   Đồng bộ hóa filters...")

            def sync_dropdowns(target_widgets):
                target_ctdl, target_nmtd, target_year = (
                    target_widgets["ctdl"],
                    target_widgets["nmtd"],
                    target_widgets["year"],
                )
                target_month = target_widgets.get("month")
                target_day = target_widgets.get("day")
                source_ctdl_val = (
                    source_widgets.get("ctdl").value
                    if source_widgets.get("ctdl")
                    else None
                )
                source_nmtd_val = (
                    source_widgets.get("nmtd").value
                    if source_widgets.get("nmtd")
                    else None
                )
                source_year_val = (
                    source_widgets.get("year").value
                    if source_widgets.get("year")
                    else None
                )
                source_month_val = (
                    source_widgets.get("month").value
                    if source_widgets.get("month")
                    else None
                )
                if source_ctdl_val is not None:
                    target_ctdl.value = source_ctdl_val
                update_nmtd(target_ctdl.value, target_nmtd)
                if (
                    source_nmtd_val is not None
                    and source_nmtd_val in target_nmtd.options
                ):
                    target_nmtd.value = source_nmtd_val
                elif target_nmtd.options:
                    target_nmtd.value = target_nmtd.options[0]
                if source_year_val is not None:
                    target_year.value = source_year_val
                if target_month and source_month_val is not None:
                    target_month.value = source_month_val
                if target_day:
                    update_day_options()

            # --- Đồng bộ hóa các tab không phải là tab đích ---
            all_widgets = {
                0: {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                },
                1: {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                },
                2: {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                },
            }
            for idx, target_widgets_dict in all_widgets.items():
                if idx != new_tab_index:
                    sync_dropdowns(target_widgets_dict)
            # --- Kích hoạt observer và nội dung tab mới ---
            print(f"   Kích hoạt nội dung cho tab {new_tab_index}...")
            if new_tab_index == 0:
                enable_observe(month_plot_widgets)
                on_change_month_plot(None)
            elif new_tab_index == 1:
                enable_observe(day_plot_widgets)
                on_change_day_plot(None)
            elif new_tab_index == 2:
                enable_observe(stat_widgets)
                on_change_stat(None)
            current_tab_index_holder[0] = new_tab_index  # Cập nhật index sau khi xử lý xong

    tabs.observe(on_tab_change, names="selected_index")

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs, out)

    # ========== KÍCH HOẠT BAN ĐẦU (Trực tiếp) ==========
    print("\n🚀 Khởi tạo nội dung cho tab mặc định...")
    initial_tab_index = tabs.selected_index
    # Kích hoạt observer và gọi hàm vẽ cho tab đầu tiên
    if initial_tab_index == 0:
        enable_observe(month_plot_widgets)
        on_change_month_plot(None)
    elif initial_tab_index == 1:
        enable_observe(day_plot_widgets)
        on_change_day_plot(None)
    elif initial_tab_index == 2:
        enable_observe(stat_widgets)
        on_change_stat(None)

elif df_all.empty:
    print("\n❌ Không có dữ liệu đầu vào từ file Parquet.")
else:
    print("\n❌ Lỗi tiền xử lý hoặc không có dữ liệu hợp lệ.")

🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 10.60 giây

🔄 Bắt đầu tiền xử lý và tối ưu hóa...
   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.
   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.
⏱️ Thời gian tiền xử lý: 139.56 giây

🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()


🚀 Khởi tạo nội dung cho tab mặc định...


In [11]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import calendar
import numpy as np
import time  # Thêm để đo thời gian (tùy chọn)

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
print("🔄 Bắt đầu đọc dữ liệu Parquet...")
start_read_time = time.time()  # Đo thời gian đọc
parquet_folder = r"C:\Khue\TDN\data\processed"

try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [
        os.path.join(parquet_folder, f)
        for f in all_files_in_folder
        if f.startswith("sanluong_") and f.endswith(".parquet")
    ]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
        df_all = pd.DataFrame()
    else:
        df_list = []
        print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
        for i, f in enumerate(parquet_files):
            try:
                df_temp = pd.read_parquet(f, engine="pyarrow")
                # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                if all(col in df_temp.columns for col in required_cols_read):
                    df_list.append(df_temp[required_cols_read])  # Chỉ lấy cột cần thiết
                else:
                    print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
            except Exception as e:
                print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

        if df_list:
            print("   Ghép các DataFrame...")
            df_all = pd.concat(df_list, ignore_index=True)
            print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
            print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
            print(f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây")
            # Hiển thị thông tin bộ nhớ (tùy chọn)
            # df_all.info(memory_usage='deep')
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame()

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame()
except Exception as e:
    print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
    df_all = pd.DataFrame()


# ==================== CELL 2: TIỀN XỬ LÝ VÀ TỐI ƯU HÓA ====================
df_sanluong = pd.DataFrame()
df_sanluong_indexed = pd.DataFrame()

if not df_all.empty:
    print("\n🔄 Bắt đầu tiền xử lý và tối ưu hóa...")
    start_process_time = time.time()
    try:
        # 1. Đổi tên cột và đảm bảo kiểu dữ liệu datetime
        df_sanluong = df_all.rename(
            columns={"ENDTIME": "TIME"}, errors="raise"
        )  # Đảm bảo cột tồn tại
        df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])

        # 2. Đảm bảo kiểu dữ liệu số cho CS và xử lý NaN
        df_sanluong["CS"] = pd.to_numeric(df_sanluong["CS"], errors="coerce")
        initial_rows = len(df_sanluong)
        df_sanluong.dropna(subset=["CS"], inplace=True)
        removed_rows = initial_rows - len(df_sanluong)
        if removed_rows > 0:
            print(f"   ⚠️ Đã loại bỏ {removed_rows:,} dòng có giá trị CS không hợp lệ.")

        # 3. Tạo các cột thời gian cần thiết
        df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year.astype(
            "int16"
        )  # Tối ưu kiểu int
        df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month.astype("int8")
        df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day.astype("int8")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME"].dt.strftime("%H:%M")

        # 4. Tối ưu kiểu dữ liệu Categorical cho các cột chuỗi lặp lại
        df_sanluong["CTDL"] = df_sanluong["CTDL"].astype("category")
        df_sanluong["NMTD"] = df_sanluong["NMTD"].astype("category")
        df_sanluong["MADIEMDO"] = df_sanluong["MADIEMDO"].astype("category")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME_SLOT"].astype("category")

        # df_sanluong bây giờ chứa tất cả dữ liệu cần thiết với kiểu tối ưu
        print("   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.")
        # df_sanluong.info(memory_usage='deep') # Kiểm tra lại bộ nhớ

        # 5. Tạo DataFrame được Index cho tính toán thống kê nhanh
        index_cols = ["CTDL", "NMTD", "YEAR", "MONTH_NUM", "DAY_NUM"]
        # Chỉ cần TIME_SLOT và CS cho df_indexed
        df_sanluong_indexed = df_sanluong[index_cols + ["TIME_SLOT", "CS"]].copy()
        # Giữ kiểu category cho index để tiết kiệm bộ nhớ index
        df_sanluong_indexed.set_index(index_cols, inplace=True)
        df_sanluong_indexed.sort_index(inplace=True)  # Rất quan trọng cho hiệu năng loc
        print("   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.")
        # df_sanluong_indexed.info(memory_usage='deep')

        print(f"⏱️ Thời gian tiền xử lý: {time.time() - start_process_time:.2f} giây")

    except Exception as e:
        print(f"❌ Lỗi trong quá trình tiền xử lý: {e}")
        # Đảm bảo cả hai df đều rỗng nếu có lỗi
        df_sanluong = pd.DataFrame()
        df_sanluong_indexed = pd.DataFrame()
else:
    print("ℹ️ Không có dữ liệu đầu vào để xử lý.")

# ==================== CELL 3: GIAO DIỆN TƯƠNG TÁC VÀ VẼ BIỂU ĐỒ ====================

# --- Chỉ tiếp tục nếu dữ liệu đã được xử lý thành công ---
if not df_sanluong.empty and not df_sanluong_indexed.empty:
    print("\n🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN (Lấy từ categories để nhanh hơn) ==========
    try:
        ctdl_list = sorted(df_sanluong["CTDL"].cat.categories)
        year_list = sorted(df_sanluong["YEAR"].unique())  # Year là số, dùng unique
        # Lấy danh sách NMTD ban đầu (tất cả) - sẽ được lọc sau
        nmtd_list_all = sorted(df_sanluong["NMTD"].cat.categories)
    except Exception as e:
        print(f"❌ Lỗi lấy danh sách dropdown: {e}")
        ctdl_list, year_list, nmtd_list_all = [], [], []

    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
        safe_options = list(options) if options is not None else []
        # Thêm giá trị None vào đầu nếu muốn có lựa chọn trống (tùy chọn)
        # safe_options = [(None, '--Chọn--')] + safe_options
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            value=safe_options[0] if safe_options else None,  # Chọn giá trị đầu tiên
            layout=widgets.Layout(width=width, margin=margin),
            style={"description_width": "auto"},
            disabled=not bool(safe_options),
        )

    # ========== DROPDOWNS ==========
    # (Tạo dropdown như cũ, sử dụng ctdl_list, year_list)
    # Tab 1
    ctdl_dropdown_month_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_month_plot = make_dropdown([], "Nhà máy:", "300px")  # Bắt đầu rỗng
    year_dropdown_month_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_month_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    placeholder_month_plot = widgets.Label(
        value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
    )
    # Tab 2
    ctdl_dropdown_day_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_day_plot = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_day_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_day_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    day_dropdown_day_plot = make_dropdown([], "Ngày:", "120px")
    # Tab 3
    ctdl_dropdown_stat = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_stat = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_stat = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_stat = make_dropdown(list(range(1, 13)), "Tháng:", "120px")

    # ========== NHÓM WIDGET ==========
    month_plot_widgets = [
        ctdl_dropdown_month_plot,
        nmtd_dropdown_month_plot,
        year_dropdown_month_plot,
        month_dropdown_month_plot,
    ]
    day_plot_widgets = [
        ctdl_dropdown_day_plot,
        nmtd_dropdown_day_plot,
        year_dropdown_day_plot,
        month_dropdown_day_plot,
        day_dropdown_day_plot,
    ]
    stat_widgets = [
        ctdl_dropdown_stat,
        nmtd_dropdown_stat,
        year_dropdown_stat,
        month_dropdown_stat,
    ]

    # ========== CẬP NHẬT NHÀ MÁY (Tối ưu: Lọc trên index category) ==========
    # Cache để tránh lọc lại NMTD cho cùng CTDL nhiều lần
    nmtd_cache = {}

    def update_nmtd(ctdl_value, dropdown_to_update):
        global nmtd_cache
        options = []
        current_nmtd_value = dropdown_to_update.value

        if ctdl_value:
            if ctdl_value in nmtd_cache:  # Kiểm tra cache trước
                options = nmtd_cache[ctdl_value]
            elif (
                not df_sanluong_indexed.empty
            ):  # Chỉ lọc nếu chưa có trong cache và df tồn tại
                try:
                    # Lấy NMTD categories từ index con tương ứng với CTDL
                    # Điều này nhanh hơn .loc[ctdl_value].index... trên df lớn
                    nmtd_options = (
                        df_sanluong_indexed.loc[ctdl_value]
                        .index.get_level_values("NMTD")
                        .unique()
                        .tolist()
                    )
                    options = sorted(nmtd_options)
                    nmtd_cache[ctdl_value] = options  # Lưu vào cache
                except KeyError:
                    options = []  # CTDL không có trong index
                except Exception as e:
                    print(f"Lỗi cập nhật NMTD cache cho '{ctdl_value}': {e}")
                    options = []

        # Cập nhật dropdown như cũ
        dropdown_to_update.options = options
        dropdown_to_update.disabled = not bool(options)
        if current_nmtd_value in options:
            dropdown_to_update.value = current_nmtd_value
        elif options:
            dropdown_to_update.value = options[0]
        else:
            dropdown_to_update.value = None

    # Gắn observe và khởi tạo NMTD (như cũ)
    ctdl_dropdown_month_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_month_plot),
        names="value",
    )
    ctdl_dropdown_day_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_day_plot), names="value"
    )
    ctdl_dropdown_stat.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_stat), names="value"
    )
    # Khởi tạo
    if ctdl_dropdown_month_plot.value:
        update_nmtd(ctdl_dropdown_month_plot.value, nmtd_dropdown_month_plot)
    if ctdl_dropdown_day_plot.value:
        update_nmtd(ctdl_dropdown_day_plot.value, nmtd_dropdown_day_plot)
    if ctdl_dropdown_stat.value:
        update_nmtd(ctdl_dropdown_stat.value, nmtd_dropdown_stat)

    # ========== GIỚI HẠN NGÀY (Như cũ) ==========
    def update_day_options(*args):
        year = year_dropdown_day_plot.value
        month = month_dropdown_day_plot.value
        current_day = day_dropdown_day_plot.value
        day_options = []
        if year and month:
            try:
                max_day = calendar.monthrange(year, month)[1]
                day_options = list(range(1, max_day + 1))
            except Exception as e:
                print(f"Lỗi lấy số ngày {month}/{year}: {e}")
        day_dropdown_day_plot.options = day_options
        day_dropdown_day_plot.disabled = not bool(day_options)
        if current_day in day_options:
            day_dropdown_day_plot.value = current_day
        elif day_options:
            day_dropdown_day_plot.value = 1
        else:
            day_dropdown_day_plot.value = None

    year_dropdown_day_plot.observe(update_day_options, names="value")
    month_dropdown_day_plot.observe(update_day_options, names="value")
    if year_dropdown_day_plot.value and month_dropdown_day_plot.value:
        update_day_options()

    # ========== VÙNG OUTPUT CHUNG ==========
    out = widgets.Output()

    # ========== VẼ BIỂU ĐỒ THEO MADIEMDO (Tab 1 & 2 - Tối ưu lọc) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        with out:
            clear_output(wait=True)
            plot_start_time = time.time()

            # Kiểm tra đầu vào
            required_vals = [ctdl, nmtd, year, month]
            if mode == "day":
                required_vals.append(day)
            if not all(v is not None for v in required_vals):
                print("   ⚠️ Vui lòng chọn đủ các bộ lọc.")
                return

            filtered_df = pd.DataFrame()
            try:
                # *** Tối ưu lọc: Lọc trên df_sanluong đã tối ưu kiểu dữ liệu ***
                # Tạo mask boolean hiệu quả
                mask = (
                    (df_sanluong["CTDL"] == ctdl)
                    & (df_sanluong["NMTD"] == nmtd)
                    & (df_sanluong["YEAR"] == year)
                    & (df_sanluong["MONTH_NUM"] == month)
                )
                if mode == "day":
                    mask &= df_sanluong["DAY_NUM"] == day

                # Chỉ lấy các cột cần thiết cho vẽ (`TIME`, `CS`, `MADIEMDO`) sau khi lọc
                filtered_df = df_sanluong.loc[mask, ["TIME", "CS", "MADIEMDO"]].copy()
                # Sắp xếp theo TIME là cần thiết cho biểu đồ đường
                filtered_df.sort_values("TIME", inplace=True)

            except Exception as e:
                print(f"❌ Lỗi khi lọc dữ liệu cho biểu đồ: {e}")

            if filtered_df.empty:
                print("   ")
                return

            # --- Vẽ biểu đồ (Giữ cấu hình gốc, thêm webgl) ---
            try:
                title_ext = (
                    f"{month}/{year}" if mode == "month" else f"{day}/{month}/{year}"
                )
                fig = px.line(
                    filtered_df,
                    x="TIME",
                    y="CS",
                    color="MADIEMDO",
                    render_mode="webgl",  # Quan trọng cho nhiều điểm
                    markers=True if mode == "day" else False,
                )
                # Áp dụng layout gốc
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.update_traces(
                    line=dict(width=2),
                    hovertemplate="Công suất: %{y}",
                )

                fig.show()

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ Plotly Express: {e}")

    # ========== VẼ BIỂU ĐỒ THỐNG KÊ THEO KHUNG GIỜ (Tab 3 - Tối ưu lọc) ==========
    def plot_timeslot_stats(ctdl, nmtd, year, month):
        with out:
            clear_output(wait=True)

            if not all([ctdl, nmtd, year, month]):
                print("   ⚠️ Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return

            stats_by_time = pd.DataFrame()
            try:
                # *** Tối ưu lọc: Dùng .loc trên df_sanluong_indexed ***
                idx_query = (ctdl, nmtd, year, month)
                # Kiểm tra sự tồn tại của index hiệu quả hơn
                if idx_query in df_sanluong_indexed.index.droplevel("DAY_NUM").unique():
                    # .loc trực tiếp trên index đã sắp xếp là rất nhanh
                    monthly_data = df_sanluong_indexed.loc[
                        idx_query, ["TIME_SLOT", "CS"]
                    ]

                    if not monthly_data.empty:
                        # Group và Agg - bước này thường nhanh trên dữ liệu đã lọc
                        stats_by_time = monthly_data.groupby(
                            "TIME_SLOT", observed=True
                        )["CS"].agg(  # observed=True tăng tốc nếu TIME_SLOT là category
                            min="min",
                            p25=lambda x: x.quantile(0.25),
                            p50="mean",
                            p75=lambda x: x.quantile(0.75),
                            max="max",
                        )
                        # Sắp xếp index (TIME_SLOT) nếu chưa đúng thứ tự (thường thì groupby giữ nguyên)
                        stats_by_time = stats_by_time.sort_index()
                    else:
                        print("   ℹ️ Không có dữ liệu chi tiết sau khi lọc.")
                else:
                    print(f"   ℹ️ Không tìm thấy dữ liệu index cho: {idx_query}")

            except KeyError:
                print("   ℹ️ Không tìm thấy dữ liệu (KeyError).")
            except Exception as e:
                print(f"❌ Lỗi tính toán thống kê: {e}")

            if stats_by_time.empty:
                print("   ℹ️ Không có dữ liệu cho lựa chọn này!")
                return

            # --- Vẽ biểu đồ (Giữ nguyên cấu hình Plotly GO) ---
            try:
                fig = go.Figure()
                time_slots = stats_by_time.index
                p25 = stats_by_time["p25"]
                p75 = stats_by_time["p75"]
                p50 = stats_by_time["p50"]
                min_cs = stats_by_time["min"]
                max_cs = stats_by_time["max"]

                # 1. Vùng Min-Max (Màu xám nhạt giống ảnh)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=max_cs,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",  # Không hiện trong legend, không hover
                        hovertemplate='Max: %{y:.2f}<extra></extra>' # Bỏ nếu không cần hover max
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=min_cs,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(211, 211, 211, 0.5)",  # Light Gray
                        name="Min-Max",  # Tên trong legend
                        hoverinfo="skip",  # Chỉ hiển thị hover ở trace dưới nếu muốn
                        hovertemplate='Min: %{y:.2f}<extra></extra>' # Bỏ nếu không cần hover min
                    )
                )

                # 2. Vùng P25-P75 (Màu xanh nhạt giống ảnh)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p75,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",
                        hovertemplate='P75: %{y:.2f}<extra></extra>' # Bỏ nếu không cần hover P75
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p25,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(173, 216, 230, 0.6)",  # Light Blue
                        name="Tần suất 25-75",  # Tên trong legend
                        hoverinfo="skip",
                        hovertemplate='P25: %{y:.2f}<extra></extra>' # Bỏ nếu không cần hover P25
                        # hovertemplate='Khung giờ: %{x}<br>P25: %{y:.2f}<extra></extra>'
                    )
                )

                # 3. Đường Median (P50) (Màu xanh dương đậm)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p50,
                        mode="lines+markers",
                        marker=dict(size=6, color="blue"),  # Màu xanh dương đậm
                        line=dict(color="blue", width=2.5),  # Dày hơn chút
                        name="Trung bình",  # Tên trong legend
                        hovertemplate="Mean: %{y:.2f}<extra></extra>",
                    )
                )

                # Layout
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Thống kê công suất theo khung giờ - {nmtd} - {month}/{year}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        tickmode="array",
                        tickvals=time_slots[:],  # Tất cả các khung giờ
                        ticktext=[t for t in time_slots],
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                        tickangle=-90,  # Xoay nhãn nếu cần
                    ),
                    yaxis=dict(
                        rangemode="tozero",
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Khung giờ</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.18,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.show()

            except Exception as e:
                print(f"❌ Lỗi vẽ biểu đồ thống kê Plotly GO: {e}")

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN (Giữ nguyên) ==========
    def on_change_month_plot(change):
        if tabs.selected_index == 0 and all(
            w.value is not None for w in month_plot_widgets
        ):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_filtered(
                    "month",
                    ctdl_dropdown_month_plot.value,
                    nmtd_dropdown_month_plot.value,
                    year_dropdown_month_plot.value,
                    month_dropdown_month_plot.value,
                )

    def on_change_day_plot(change):
        if tabs.selected_index == 1 and all(
            w.value is not None for w in day_plot_widgets
        ):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_filtered(
                    "day",
                    ctdl_dropdown_day_plot.value,
                    nmtd_dropdown_day_plot.value,
                    year_dropdown_day_plot.value,
                    month_dropdown_day_plot.value,
                    day_dropdown_day_plot.value,
                )

    def on_change_stat(change):
        if tabs.selected_index == 2 and all(w.value is not None for w in stat_widgets):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_timeslot_stats(
                    ctdl_dropdown_stat.value,
                    nmtd_dropdown_stat.value,
                    year_dropdown_stat.value,
                    month_dropdown_stat.value,
                )

    # ========== HÀM TẮT / BẬT OBSERVER (Giữ nguyên) ==========
    _observed_callbacks = {}

    def disable_observe(widgets_list):
        for w in widgets_list:
            if w in _observed_callbacks and _observed_callbacks[w]["active"]:
                try:
                    w.unobserve(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = False
                except:
                    _observed_callbacks[w]["active"] = (
                        False  # Ignore errors during unobserve
                    )

    def enable_observe(widgets_list):
        for w in widgets_list:
            if (
                w in _observed_callbacks
                and not w.disabled
                and not _observed_callbacks[w]["active"]
            ):
                try:
                    w.observe(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = True
                except Exception as e:
                    print(f"Lỗi observe {w.description}: {e}")

    # ========== GẮN OBSERVER BAN ĐẦU (Giữ nguyên) ==========
    for w in month_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_month_plot, "active": False}
    for w in day_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_day_plot, "active": False}
    for w in stat_widgets:
        _observed_callbacks[w] = {"func": on_change_stat, "active": False}

    # ========== TẠO GIAO DIỆN TABS (Giữ nguyên) ==========
    tab_month_plot = widgets.HBox(
        month_plot_widgets + [placeholder_month_plot],
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_day_plot = widgets.HBox(
        day_plot_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_stat = widgets.HBox(
        stat_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tabs = widgets.Tab(children=[tab_month_plot, tab_day_plot, tab_stat])
    tabs.set_title(0, "📅 Biểu đồ tháng")
    tabs.set_title(1, "🗓️ Biểu đồ ngày")
    tabs.set_title(2, "📊 Thống kê tháng")

    # ========== XỬ LÝ CHUYỂN TAB (Đồng bộ hóa - Sửa lại current_tab_index) ==========
    # Sử dụng biến này trong scope của cell, không cần global
    current_tab_index = tabs.selected_index

# ========== XỬ LÝ CHUYỂN TAB (Đồng bộ hóa - Sửa lại) ==========
# Không cần biến current_tab_index nữa vì ta sẽ sử dụng change["old"]
    def on_tab_change(change):
        if change["name"] == "selected_index":
            new_tab_index = change["new"]
            old_tab_index = change["old"]  # Sử dụng giá trị old từ thay đổi

            # --- Tắt observer tab cũ ---
            if old_tab_index == 0:
                disable_observe(month_plot_widgets)
            elif old_tab_index == 1:
                disable_observe(day_plot_widgets)
            elif old_tab_index == 2:
                disable_observe(stat_widgets)

            # --- Xác định nguồn đồng bộ ---
            source_widgets = {}  # Dictionary để lưu widget nguồn
            if old_tab_index == 0:
                source_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }
            elif old_tab_index == 1:
                source_widgets = {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                }
            elif old_tab_index == 2:
                source_widgets = {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                }
            else:  # Khởi tạo
                source_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }

            # --- Xác định widgets đích của tab mới ---
            target_widgets = {}
            if new_tab_index == 0:
                target_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }
            elif new_tab_index == 1:
                target_widgets = {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                }
            elif new_tab_index == 2:
                target_widgets = {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                }

            # --- Đồng bộ từ nguồn sang đích ---
            if old_tab_index != new_tab_index and old_tab_index is not None:
                # Đồng bộ CTDL
                if source_widgets.get("ctdl") and source_widgets["ctdl"].value is not None:
                    target_widgets["ctdl"].value = source_widgets["ctdl"].value
                
                # Đồng bộ NMTD (phải cập nhật options trước)
                target_nmtd = target_widgets["nmtd"]
                update_nmtd(target_widgets["ctdl"].value, target_nmtd)
                if source_widgets.get("nmtd") and source_widgets["nmtd"].value in target_nmtd.options:
                    target_nmtd.value = source_widgets["nmtd"].value
                
                # Đồng bộ Year
                if source_widgets.get("year") and source_widgets["year"].value is not None:
                    target_widgets["year"].value = source_widgets["year"].value
                
                # Đồng bộ Month
                if source_widgets.get("month") and source_widgets["month"].value is not None:
                    target_widgets["month"].value = source_widgets["month"].value
                
                # Nếu chuyển sang tab ngày, cập nhật dropdown ngày
                if "day" in target_widgets:
                    update_day_options()  # Cập nhật options ngày dựa trên năm/tháng đã chọn
                    # Mặc định chọn ngày 1 nếu đến từ tab khác
                    if old_tab_index != 1 and target_widgets["day"].options:
                        target_widgets["day"].value = 1

            # --- Kích hoạt observer và cập nhật nội dung tab mới ---
            if new_tab_index == 0:
                enable_observe(month_plot_widgets)
                on_change_month_plot(None)  # Gọi hàm vẽ/tính toán tương ứng
            elif new_tab_index == 1:
                enable_observe(day_plot_widgets)
                on_change_day_plot(None)
            elif new_tab_index == 2:
                enable_observe(stat_widgets)
                on_change_stat(None)

    tabs.observe(on_tab_change, names="selected_index")

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs, out)

    # ========== KÍCH HOẠT BAN ĐẦU ==========
    # Kích hoạt lần đầu bằng cách gọi on_tab_change mô phỏng với old=None
    on_tab_change(
        {
            "name": "selected_index",
            "old": None,
            "new": tabs.selected_index,
            "owner": tabs,
            "type": "change",
        }
    )
elif df_all.empty:
    print("\n❌ Không có dữ liệu đầu vào từ file Parquet.")
else:  # df_sanluong hoặc df_sanluong_indexed rỗng
    print("\n❌ Lỗi tiền xử lý hoặc không có dữ liệu hợp lệ.")


🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 14.24 giây

🔄 Bắt đầu tiền xử lý và tối ưu hóa...
   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.
   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.
⏱️ Thời gian tiền xử lý: 128.34 giây

🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import calendar
import numpy as np
import time  # Thêm để đo thời gian (tùy chọn)

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
print("🔄 Bắt đầu đọc dữ liệu Parquet...")
start_read_time = time.time()  # Đo thời gian đọc
parquet_folder = r"C:\Khue\TDN\data\processed"

try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [
        os.path.join(parquet_folder, f)
        for f in all_files_in_folder
        if f.startswith("sanluong_") and f.endswith(".parquet")
    ]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
        df_all = pd.DataFrame()
    else:
        df_list = []
        print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
        for i, f in enumerate(parquet_files):
            try:
                df_temp = pd.read_parquet(f, engine="pyarrow")
                # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                if all(col in df_temp.columns for col in required_cols_read):
                    df_list.append(df_temp[required_cols_read])  # Chỉ lấy cột cần thiết
                else:
                    print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
            except Exception as e:
                print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

        if df_list:
            print("   Ghép các DataFrame...")
            df_all = pd.concat(df_list, ignore_index=True)
            print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
            print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
            print(f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây")
            # Hiển thị thông tin bộ nhớ (tùy chọn)
            # df_all.info(memory_usage='deep')
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame()

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame()
except Exception as e:
    print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
    df_all = pd.DataFrame()


# ==================== CELL 2: TIỀN XỬ LÝ VÀ TỐI ƯU HÓA ====================
df_sanluong = pd.DataFrame()
df_sanluong_indexed = pd.DataFrame()

if not df_all.empty:
    print("\n🔄 Bắt đầu tiền xử lý và tối ưu hóa...")
    start_process_time = time.time()
    try:
        # 1. Đổi tên cột và đảm bảo kiểu dữ liệu datetime
        df_sanluong = df_all.rename(
            columns={"ENDTIME": "TIME"}, errors="raise"
        )  # Đảm bảo cột tồn tại
        df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])

        # 2. Đảm bảo kiểu dữ liệu số cho CS và xử lý NaN
        df_sanluong["CS"] = pd.to_numeric(df_sanluong["CS"], errors="coerce")
        initial_rows = len(df_sanluong)
        df_sanluong.dropna(subset=["CS"], inplace=True)
        removed_rows = initial_rows - len(df_sanluong)
        if removed_rows > 0:
            print(f"   ⚠️ Đã loại bỏ {removed_rows:,} dòng có giá trị CS không hợp lệ.")

        # 3. Tạo các cột thời gian cần thiết
        df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year.astype(
            "int16"
        )  # Tối ưu kiểu int
        df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month.astype("int8")
        df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day.astype("int8")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME"].dt.strftime("%H:%M")

        # 4. Tối ưu kiểu dữ liệu Categorical cho các cột chuỗi lặp lại
        df_sanluong["CTDL"] = df_sanluong["CTDL"].astype("category")
        df_sanluong["NMTD"] = df_sanluong["NMTD"].astype("category")
        df_sanluong["MADIEMDO"] = df_sanluong["MADIEMDO"].astype("category")
        df_sanluong["TIME_SLOT"] = df_sanluong["TIME_SLOT"].astype("category")

        # df_sanluong bây giờ chứa tất cả dữ liệu cần thiết với kiểu tối ưu
        print("   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.")
        # df_sanluong.info(memory_usage='deep') # Kiểm tra lại bộ nhớ

        # 5. Tạo DataFrame được Index cho tính toán thống kê nhanh
        index_cols = ["CTDL", "NMTD", "MADIEMDO","YEAR", "MONTH_NUM", "DAY_NUM"]
        # Chỉ cần TIME_SLOT và CS cho df_indexed
        df_sanluong_indexed = df_sanluong[index_cols + ["TIME_SLOT", "CS"]].copy()
        # Giữ kiểu category cho index để tiết kiệm bộ nhớ index
        df_sanluong_indexed.set_index(index_cols, inplace=True)
        df_sanluong_indexed.sort_index(inplace=True)  # Rất quan trọng cho hiệu năng loc
        print("   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.")
        # df_sanluong_indexed.info(memory_usage='deep')

        print(f"⏱️ Thời gian tiền xử lý: {time.time() - start_process_time:.2f} giây")

    except Exception as e:
        print(f"❌ Lỗi trong quá trình tiền xử lý: {e}")
        # Đảm bảo cả hai df đều rỗng nếu có lỗi
        df_sanluong = pd.DataFrame()
        df_sanluong_indexed = pd.DataFrame()
else:
    print("ℹ️ Không có dữ liệu đầu vào để xử lý.")

# ==================== CELL 3: GIAO DIỆN TƯƠNG TÁC VÀ VẼ BIỂU ĐỒ ====================

# --- Chỉ tiếp tục nếu dữ liệu đã được xử lý thành công ---
if not df_sanluong.empty and not df_sanluong_indexed.empty:
    print("\n🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN (Lấy từ categories để nhanh hơn) ==========
    try:
        ctdl_list = sorted(df_sanluong["CTDL"].cat.categories)
        year_list = sorted(df_sanluong["YEAR"].unique())  # Year là số, dùng unique
        # Lấy danh sách NMTD ban đầu (tất cả) - sẽ được lọc sau
        nmtd_list_all = sorted(df_sanluong["NMTD"].cat.categories)
    except Exception as e:
        print(f"❌ Lỗi lấy danh sách dropdown: {e}")
        ctdl_list, year_list, nmtd_list_all = [], [], []

    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
        safe_options = list(options) if options is not None else []
        # Thêm giá trị None vào đầu nếu muốn có lựa chọn trống (tùy chọn)
        # safe_options = [(None, '--Chọn--')] + safe_options
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            value=safe_options[0] if safe_options else None,  # Chọn giá trị đầu tiên
            layout=widgets.Layout(width=width, margin=margin),
            style={"description_width": "auto"},
            disabled=not bool(safe_options),
        )

    # ========== DROPDOWNS ==========
    # (Tạo dropdown như cũ, sử dụng ctdl_list, year_list)
    # Tab 1
    ctdl_dropdown_month_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_month_plot = make_dropdown([], "Nhà máy:", "300px")  # Bắt đầu rỗng
    year_dropdown_month_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_month_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    placeholder_month_plot = widgets.Label(
        value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
    )
    # Tab 2
    ctdl_dropdown_day_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_day_plot = make_dropdown([], "Nhà máy:", "300px")
    year_dropdown_day_plot = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_day_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    day_dropdown_day_plot = make_dropdown([], "Ngày:", "120px")
    # Tab 3
    ctdl_dropdown_stat = make_dropdown(ctdl_list, "CTDL:", "300px")
    nmtd_dropdown_stat = make_dropdown([], "Nhà máy:", "300px")
    madiemdo_dropdown_stat = make_dropdown([], "Mã điểm đo:", "300px")
    year_dropdown_stat = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_stat = make_dropdown(list(range(1, 13)), "Tháng:", "120px")

    # ========== NHÓM WIDGET ==========
    month_plot_widgets = [
        ctdl_dropdown_month_plot,
        nmtd_dropdown_month_plot,
        year_dropdown_month_plot,
        month_dropdown_month_plot,
    ]
    day_plot_widgets = [
        ctdl_dropdown_day_plot,
        nmtd_dropdown_day_plot,
        year_dropdown_day_plot,
        month_dropdown_day_plot,
        day_dropdown_day_plot,
    ]
    stat_widgets = [
        ctdl_dropdown_stat,
        nmtd_dropdown_stat,
        madiemdo_dropdown_stat,
        year_dropdown_stat,
        month_dropdown_stat,
    ]

    # ========== CẬP NHẬT NHÀ MÁY (Tối ưu: Lọc trên index category) ==========
    # Cache để tránh lọc lại NMTD cho cùng CTDL nhiều lần
    nmtd_cache = {}
    madiemdo_cache = {}

    def update_nmtd(ctdl_value, dropdown_to_update):
        global nmtd_cache
        options = []
        current_nmtd_value = dropdown_to_update.value

        if ctdl_value:
            if ctdl_value in nmtd_cache:  # Kiểm tra cache trước
                options = nmtd_cache[ctdl_value]
            elif (
                not df_sanluong_indexed.empty
            ):  # Chỉ lọc nếu chưa có trong cache và df tồn tại
                try:
                    # Lấy NMTD categories từ index con tương ứng với CTDL
                    # Điều này nhanh hơn .loc[ctdl_value].index... trên df lớn
                    nmtd_options = (
                        df_sanluong_indexed.loc[ctdl_value]
                        .index.get_level_values("NMTD")
                        .unique()
                        .tolist()
                    )
                    options = sorted(nmtd_options)
                    nmtd_cache[ctdl_value] = options  # Lưu vào cache
                except KeyError:
                    options = []  # CTDL không có trong index
                except Exception as e:
                    print(f"Lỗi cập nhật NMTD cache cho '{ctdl_value}': {e}")
                    options = []

        # Cập nhật dropdown như cũ
        dropdown_to_update.options = options
        dropdown_to_update.disabled = not bool(options)
        if current_nmtd_value in options:
            dropdown_to_update.value = current_nmtd_value
        elif options:
            dropdown_to_update.value = options[0]
        else:
            dropdown_to_update.value = None

    def update_madiemdo(ctdl_value, nmtd_value, dropdown_to_update):
        global madiemdo_cache
        options = []
        current_madiemdo_value = dropdown_to_update.value

        if ctdl_value and nmtd_value:
            cache_key = f"{ctdl_value}_{nmtd_value}"
            if cache_key in madiemdo_cache:  # Kiểm tra cache trước
                options = madiemdo_cache[cache_key]
            elif not df_sanluong.empty:  # Chỉ lọc nếu chưa có trong cache
                try:
                    # Lấy danh sách các mã điểm đo của nhà máy đã chọn
                    madiemdo_options = (
                        df_sanluong[
                            (df_sanluong["CTDL"] == ctdl_value)
                            & (df_sanluong["NMTD"] == nmtd_value)
                        ]["MADIEMDO"]
                        .unique()
                        .tolist()
                    )

                    options = sorted(madiemdo_options)
                    madiemdo_cache[cache_key] = options  # Lưu vào cache
                except Exception as e:
                    print(f"Lỗi cập nhật MADIEMDO cache cho '{cache_key}': {e}")
                    options = []

        # Cập nhật dropdown
        dropdown_to_update.options = options
        dropdown_to_update.disabled = not bool(options)
        if current_madiemdo_value in options:
            dropdown_to_update.value = current_madiemdo_value
        elif options:
            dropdown_to_update.value = options[0]
        else:
            dropdown_to_update.value = None

    # Gắn observe và khởi tạo NMTD (như cũ)
    ctdl_dropdown_month_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_month_plot),
        names="value",
    )
    ctdl_dropdown_day_plot.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_day_plot), names="value"
    )
    ctdl_dropdown_stat.observe(
        lambda change: update_nmtd(change["new"], nmtd_dropdown_stat), names="value"
    )
    nmtd_dropdown_stat.observe(
        lambda change: update_madiemdo(
            ctdl_dropdown_stat.value, change["new"], madiemdo_dropdown_stat
        ),
        names="value",
    )
    # Khởi tạo
    if ctdl_dropdown_month_plot.value:
        update_nmtd(ctdl_dropdown_month_plot.value, nmtd_dropdown_month_plot)
    if ctdl_dropdown_day_plot.value:
        update_nmtd(ctdl_dropdown_day_plot.value, nmtd_dropdown_day_plot)
    if ctdl_dropdown_stat.value:
        update_nmtd(ctdl_dropdown_stat.value, nmtd_dropdown_stat)
    if ctdl_dropdown_stat.value and nmtd_dropdown_stat.value:
        update_madiemdo(
            ctdl_dropdown_stat.value, nmtd_dropdown_stat.value, madiemdo_dropdown_stat
        )

    # ========== GIỚI HẠN NGÀY (Như cũ) ==========
    def update_day_options(*args):
        year = year_dropdown_day_plot.value
        month = month_dropdown_day_plot.value
        current_day = day_dropdown_day_plot.value
        day_options = []
        if year and month:
            try:
                max_day = calendar.monthrange(year, month)[1]
                day_options = list(range(1, max_day + 1))
            except Exception as e:
                print(f"Lỗi lấy số ngày {month}/{year}: {e}")
        day_dropdown_day_plot.options = day_options
        day_dropdown_day_plot.disabled = not bool(day_options)
        if current_day in day_options:
            day_dropdown_day_plot.value = current_day
        elif day_options:
            day_dropdown_day_plot.value = 1
        else:
            day_dropdown_day_plot.value = None

    year_dropdown_day_plot.observe(update_day_options, names="value")
    month_dropdown_day_plot.observe(update_day_options, names="value")
    if year_dropdown_day_plot.value and month_dropdown_day_plot.value:
        update_day_options()

    # ========== VÙNG OUTPUT CHUNG ==========
    out = widgets.Output()

    # ========== VẼ BIỂU ĐỒ THEO MADIEMDO (Tab 1 & 2 - Tối ưu lọc) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        with out:
            clear_output(wait=True)
            plot_start_time = time.time()

            # Kiểm tra đầu vào
            required_vals = [ctdl, nmtd, year, month]
            if mode == "day":
                required_vals.append(day)
            if not all(v is not None for v in required_vals):
                print("   ⚠️ Vui lòng chọn đủ các bộ lọc.")
                return

            filtered_df = pd.DataFrame()
            try:
                # *** Tối ưu lọc: Lọc trên df_sanluong đã tối ưu kiểu dữ liệu ***
                # Tạo mask boolean hiệu quả
                mask = (
                    (df_sanluong["CTDL"] == ctdl)
                    & (df_sanluong["NMTD"] == nmtd)
                    & (df_sanluong["YEAR"] == year)
                    & (df_sanluong["MONTH_NUM"] == month)
                )
                if mode == "day":
                    mask &= df_sanluong["DAY_NUM"] == day

                # Chỉ lấy các cột cần thiết cho vẽ (`TIME`, `CS`, `MADIEMDO`) sau khi lọc
                filtered_df = df_sanluong.loc[mask, ["TIME", "CS", "MADIEMDO"]].copy()
                # Sắp xếp theo TIME là cần thiết cho biểu đồ đường
                filtered_df.sort_values("TIME", inplace=True)

            except Exception as e:
                print(f"❌ Lỗi khi lọc dữ liệu cho biểu đồ: {e}")

            if filtered_df.empty:
                print("   ")
                return

            # --- Vẽ biểu đồ (Giữ cấu hình gốc, thêm webgl) ---
            try:
                title_ext = (
                    f"{month}/{year}" if mode == "month" else f"{day}/{month}/{year}"
                )
                fig = px.line(
                    filtered_df,
                    x="TIME",
                    y="CS",
                    color="MADIEMDO",
                    render_mode="webgl",  # Quan trọng cho nhiều điểm
                    markers=True if mode == "day" else False,
                )
                # Áp dụng layout gốc
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.update_traces(
                    line=dict(width=2),
                    hovertemplate="Công suất: %{y}",
                )

                fig.show()

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ Plotly Express: {e}")

    # ========== VẼ BIỂU ĐỒ THỐNG KÊ THEO KHUNG GIỜ (Tab 3 - Tối ưu lọc) ==========
    def plot_timeslot_stats(ctdl, nmtd, madiemdo, year, month):
        with out:
            clear_output(wait=True)

            if not all([ctdl, nmtd, year, month]):
                print("   ⚠️ Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return

            stats_by_time = pd.DataFrame()
            try:
                # *** Tối ưu lọc: Dùng .loc trên df_sanluong_indexed ***
                idx_query = (ctdl, nmtd, madiemdo, year, month)
                # Kiểm tra sự tồn tại của index hiệu quả hơn
                if idx_query in df_sanluong_indexed.index.droplevel("DAY_NUM").unique():
                    # .loc trực tiếp trên index đã sắp xếp là rất nhanh
                    monthly_data = df_sanluong_indexed.loc[
                        idx_query, ["TIME_SLOT", "CS"]
                    ]

                    if not monthly_data.empty:
                        # Group và Agg - bước này thường nhanh trên dữ liệu đã lọc
                        stats_by_time = monthly_data.groupby(
                            "TIME_SLOT", observed=True
                        )["CS"].agg(  # observed=True tăng tốc nếu TIME_SLOT là category
                            min="min",
                            p25=lambda x: x.quantile(0.25),
                            p50="mean",
                            p75=lambda x: x.quantile(0.75),
                            max="max",
                        )
                        # Sắp xếp index (TIME_SLOT) nếu chưa đúng thứ tự (thường thì groupby giữ nguyên)
                        stats_by_time = stats_by_time.sort_index()
                    else:
                        print("   ℹ️ Không có dữ liệu chi tiết sau khi lọc.")

            except KeyError:
                print("   ℹ️ Không tìm thấy dữ liệu (KeyError).")
            except Exception as e:
                print(f"❌ Lỗi tính toán thống kê: {e}")

            if stats_by_time.empty:
                print("   ℹ️ Không có dữ liệu cho lựa chọn này!")
                return

            # --- Vẽ biểu đồ (Giữ nguyên cấu hình Plotly GO) ---
            try:
                fig = go.Figure()
                time_slots = stats_by_time.index
                p25 = stats_by_time["p25"]
                p75 = stats_by_time["p75"]
                p50 = stats_by_time["p50"]
                min_cs = stats_by_time["min"]
                max_cs = stats_by_time["max"]

                # 1. Vùng Min-Max (Màu xám nhạt giống ảnh)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=max_cs,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",  # Không hiện trong legend, không hover
                        hovertemplate="Max: %{y:.2f}<extra></extra>",  # Bỏ nếu không cần hover max
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=min_cs,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(211, 211, 211, 0.5)",  # Light Gray
                        name="Min-Max",  # Tên trong legend
                        hoverinfo="skip",  # Chỉ hiển thị hover ở trace dưới nếu muốn
                        hovertemplate="Min: %{y:.2f}<extra></extra>",  # Bỏ nếu không cần hover min
                    )
                )

                # 2. Vùng P25-P75 (Màu xanh nhạt giống ảnh)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p75,
                        mode="lines",
                        line=dict(width=0),
                        showlegend=False,
                        hoverinfo="skip",
                        hovertemplate="P75: %{y:.2f}<extra></extra>",  # Bỏ nếu không cần hover P75
                    )
                )
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p25,
                        mode="lines",
                        line=dict(width=0),
                        fill="tonexty",
                        fillcolor="rgba(173, 216, 230, 0.6)",  # Light Blue
                        name="Tần suất 25-75",  # Tên trong legend
                        hoverinfo="skip",
                        hovertemplate="P25: %{y:.2f}<extra></extra>",  # Bỏ nếu không cần hover P25
                        # hovertemplate='Khung giờ: %{x}<br>P25: %{y:.2f}<extra></extra>'
                    )
                )

                # 3. Đường Median (P50) (Màu xanh dương đậm)
                fig.add_trace(
                    go.Scatter(
                        x=time_slots,
                        y=p50,
                        mode="lines+markers",
                        marker=dict(size=6, color="blue"),  # Màu xanh dương đậm
                        line=dict(color="blue", width=2.5),  # Dày hơn chút
                        name="Trung bình",  # Tên trong legend
                        hovertemplate="Mean: %{y:.2f}<extra></extra>",
                    )
                )

                # Layout
                fig.update_layout(
                    title=None,
                    annotations=[
                        dict(
                            text=f"<b>Thống kê công suất theo khung giờ - {nmtd} - {month}/{year}</b>",
                            xref="paper",
                            yref="paper",
                            x=0.45,
                            y=1.1,
                            xanchor="center",
                            yanchor="top",
                            showarrow=False,
                            font=dict(size=18, color="black"),
                        )
                    ],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title=None,
                    yaxis_title="Giá trị công suất",
                    xaxis=dict(
                        tickmode="array",
                        tickvals=time_slots[:],  # Tất cả các khung giờ
                        ticktext=[t for t in time_slots],
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                        tickangle=-90,  # Xoay nhãn nếu cần
                    ),
                    yaxis=dict(
                        rangemode="tozero",
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                    ),
                    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff",
                )
                fig.add_annotation(
                    text="<b>Khung giờ</b>",
                    xref="paper",
                    yref="paper",
                    x=0.45,
                    y=-0.18,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.show()

            except Exception as e:
                print(f"❌ Lỗi vẽ biểu đồ thống kê Plotly GO: {e}")

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN (Giữ nguyên) ==========
    def on_change_month_plot(change):
        if tabs.selected_index == 0 and all(
            w.value is not None for w in month_plot_widgets
        ):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_filtered(
                    "month",
                    ctdl_dropdown_month_plot.value,
                    nmtd_dropdown_month_plot.value,
                    year_dropdown_month_plot.value,
                    month_dropdown_month_plot.value,
                )

    def on_change_day_plot(change):
        if tabs.selected_index == 1 and all(
            w.value is not None for w in day_plot_widgets
        ):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_filtered(
                    "day",
                    ctdl_dropdown_day_plot.value,
                    nmtd_dropdown_day_plot.value,
                    year_dropdown_day_plot.value,
                    month_dropdown_day_plot.value,
                    day_dropdown_day_plot.value,
                )

    def on_change_stat(change):
        if tabs.selected_index == 2 and all(w.value is not None for w in stat_widgets):
            if change is None or (
                isinstance(change, dict) and change.get("new") != change.get("old")
            ):
                plot_timeslot_stats(
                    ctdl_dropdown_stat.value,
                    nmtd_dropdown_stat.value,
                    madiemdo_dropdown_stat.value,
                    year_dropdown_stat.value,
                    month_dropdown_stat.value,
                )

    # ========== HÀM TẮT / BẬT OBSERVER (Giữ nguyên) ==========
    _observed_callbacks = {}

    def disable_observe(widgets_list):
        for w in widgets_list:
            if w in _observed_callbacks and _observed_callbacks[w]["active"]:
                try:
                    w.unobserve(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = False
                except:
                    _observed_callbacks[w]["active"] = (
                        False  # Ignore errors during unobserve
                    )

    def enable_observe(widgets_list):
        for w in widgets_list:
            if (
                w in _observed_callbacks
                and not w.disabled
                and not _observed_callbacks[w]["active"]
            ):
                try:
                    w.observe(_observed_callbacks[w]["func"], names="value")
                    _observed_callbacks[w]["active"] = True
                except Exception as e:
                    print(f"Lỗi observe {w.description}: {e}")

    # ========== GẮN OBSERVER BAN ĐẦU (Giữ nguyên) ==========
    for w in month_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_month_plot, "active": False}
    for w in day_plot_widgets:
        _observed_callbacks[w] = {"func": on_change_day_plot, "active": False}
    for w in stat_widgets:
        _observed_callbacks[w] = {"func": on_change_stat, "active": False}

    # ========== TẠO GIAO DIỆN TABS (Giữ nguyên) ==========
    tab_month_plot = widgets.HBox(
        month_plot_widgets + [placeholder_month_plot],
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_day_plot = widgets.HBox(
        day_plot_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tab_stat = widgets.HBox(
        stat_widgets,
        layout=widgets.Layout(
            justify_content="flex-start", align_items="flex-end", overflow="visible"
        ),
    )
    tabs = widgets.Tab(children=[tab_month_plot, tab_day_plot, tab_stat])
    tabs.set_title(0, "📅 Biểu đồ tháng")
    tabs.set_title(1, "🗓️ Biểu đồ ngày")
    tabs.set_title(2, "📊 Thống kê tháng")

    # ========== XỬ LÝ CHUYỂN TAB (Đồng bộ hóa - Sửa lại current_tab_index) ==========
    # Sử dụng biến này trong scope của cell, không cần global
    current_tab_index = tabs.selected_index

    # ========== XỬ LÝ CHUYỂN TAB (Đồng bộ hóa - Sửa lại) ==========
    # Không cần biến current_tab_index nữa vì ta sẽ sử dụng change["old"]
    def on_tab_change(change):
        if change["name"] == "selected_index":
            new_tab_index = change["new"]
            old_tab_index = change["old"]  # Sử dụng giá trị old từ thay đổi

            # --- Tắt observer tab cũ ---
            if old_tab_index == 0:
                disable_observe(month_plot_widgets)
            elif old_tab_index == 1:
                disable_observe(day_plot_widgets)
            elif old_tab_index == 2:
                disable_observe(stat_widgets)

            # --- Xác định nguồn đồng bộ ---
            source_widgets = {}  # Dictionary để lưu widget nguồn
            if old_tab_index == 0:
                source_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }
            elif old_tab_index == 1:
                source_widgets = {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                }
            elif old_tab_index == 2:
                source_widgets = {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                }
            else:  # Khởi tạo
                source_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }

            # --- Xác định widgets đích của tab mới ---
            target_widgets = {}
            if new_tab_index == 0:
                target_widgets = {
                    "ctdl": ctdl_dropdown_month_plot,
                    "nmtd": nmtd_dropdown_month_plot,
                    "year": year_dropdown_month_plot,
                    "month": month_dropdown_month_plot,
                }
            elif new_tab_index == 1:
                target_widgets = {
                    "ctdl": ctdl_dropdown_day_plot,
                    "nmtd": nmtd_dropdown_day_plot,
                    "year": year_dropdown_day_plot,
                    "month": month_dropdown_day_plot,
                    "day": day_dropdown_day_plot,
                }
            elif new_tab_index == 2:
                target_widgets = {
                    "ctdl": ctdl_dropdown_stat,
                    "nmtd": nmtd_dropdown_stat,
                    "madiemdo": madiemdo_dropdown_stat,
                    "year": year_dropdown_stat,
                    "month": month_dropdown_stat,
                }

            # --- Đồng bộ từ nguồn sang đích ---
            if old_tab_index != new_tab_index and old_tab_index is not None:
                # Đồng bộ CTDL
                if (
                    source_widgets.get("ctdl")
                    and source_widgets["ctdl"].value is not None
                ):
                    target_widgets["ctdl"].value = source_widgets["ctdl"].value

                # Đồng bộ NMTD (phải cập nhật options trước)
                target_nmtd = target_widgets["nmtd"]
                update_nmtd(target_widgets["ctdl"].value, target_nmtd)
                if (
                    source_widgets.get("nmtd")
                    and source_widgets["nmtd"].value in target_nmtd.options
                ):
                    target_nmtd.value = source_widgets["nmtd"].value
                # Đồng bộ MADIEMDO (phải cập nhật options trước)
                if new_tab_index == 2 and "madiemdo" in target_widgets:
                    update_madiemdo(
                        target_widgets["ctdl"].value,
                        target_widgets["nmtd"].value,
                        target_widgets["madiemdo"],
                    )

                # Đồng bộ Year
                if (
                    source_widgets.get("year")
                    and source_widgets["year"].value is not None
                ):
                    target_widgets["year"].value = source_widgets["year"].value

                # Đồng bộ Month
                if (
                    source_widgets.get("month")
                    and source_widgets["month"].value is not None
                ):
                    target_widgets["month"].value = source_widgets["month"].value

                # Nếu chuyển sang tab ngày, cập nhật dropdown ngày
                if "day" in target_widgets:
                    update_day_options()  # Cập nhật options ngày dựa trên năm/tháng đã chọn
                    # Mặc định chọn ngày 1 nếu đến từ tab khác
                    if old_tab_index != 1 and target_widgets["day"].options:
                        target_widgets["day"].value = 1

            # --- Kích hoạt observer và cập nhật nội dung tab mới ---
            if new_tab_index == 0:
                enable_observe(month_plot_widgets)
                on_change_month_plot(None)  # Gọi hàm vẽ/tính toán tương ứng
            elif new_tab_index == 1:
                enable_observe(day_plot_widgets)
                on_change_day_plot(None)
            elif new_tab_index == 2:
                enable_observe(stat_widgets)
                on_change_stat(None)

    tabs.observe(on_tab_change, names="selected_index")

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs, out)

    # ========== KÍCH HOẠT BAN ĐẦU ==========
    # Kích hoạt lần đầu bằng cách gọi on_tab_change mô phỏng với old=None
    on_tab_change(
        {
            "name": "selected_index",
            "old": None,
            "new": tabs.selected_index,
            "owner": tabs,
            "type": "change",
        }
    )
elif df_all.empty:
    print("\n❌ Không có dữ liệu đầu vào từ file Parquet.")
else:  # df_sanluong hoặc df_sanluong_indexed rỗng
    print("\n❌ Lỗi tiền xử lý hoặc không có dữ liệu hợp lệ.")


🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 19.25 giây

🔄 Bắt đầu tiền xử lý và tối ưu hóa...
   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.
   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.
⏱️ Thời gian tiền xử lý: 133.17 giây

🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import calendar
import numpy as np
import time  # Thêm để đo thời gian (tùy chọn)

# Biến toàn cục để lưu dữ liệu đã xử lý
df_all = pd.DataFrame()
df_sanluong = pd.DataFrame()
df_sanluong_indexed = pd.DataFrame()
nmtd_cache = {}
madiemdo_cache = {}
_observed_callbacks = {}
ui_components = {}
parquet_folder = r"C:\Khue\TDN\data\processed"
def read_data(parquet_folder):
    """Đọc dữ liệu từ các file Parquet"""
    global df_all

    print("🔄 Bắt đầu đọc dữ liệu Parquet...")
    start_read_time = time.time()

    try:
        all_files_in_folder = os.listdir(parquet_folder)
        parquet_files = [
            os.path.join(parquet_folder, f)
            for f in all_files_in_folder
            if f.startswith("sanluong_") and f.endswith(".parquet")
        ]

        if not parquet_files:
            print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
            df_all = pd.DataFrame()
        else:
            df_list = []
            print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
            for i, f in enumerate(parquet_files):
                try:
                    df_temp = pd.read_parquet(f, engine="pyarrow")
                    # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                    required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                    if all(col in df_temp.columns for col in required_cols_read):
                        df_list.append(
                            df_temp[required_cols_read]
                        )  # Chỉ lấy cột cần thiết
                    else:
                        print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
                except Exception as e:
                    print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

            if df_list:
                print("   Ghép các DataFrame...")
                df_all = pd.concat(df_list, ignore_index=True)
                print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
                print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
                print(
                    f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây"
                )
            else:
                print("❌ Không đọc được file nào thành công.")
                df_all = pd.DataFrame()

    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
        df_all = pd.DataFrame()
    except Exception as e:
        print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
        df_all = pd.DataFrame()

    return df_all


def preprocess_data():
    """Tiền xử lý và tối ưu hóa dữ liệu"""
    global df_all, df_sanluong, df_sanluong_indexed

    df_sanluong = pd.DataFrame()
    df_sanluong_indexed = pd.DataFrame()

    if not df_all.empty:
        print("\n🔄 Bắt đầu tiền xử lý và tối ưu hóa...")
        start_process_time = time.time()
        try:
            # 1. Đổi tên cột và đảm bảo kiểu dữ liệu datetime
            df_sanluong = df_all.rename(
                columns={"ENDTIME": "TIME"}, errors="raise"
            )  # Đảm bảo cột tồn tại
            df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])

            # Thêm bước lọc các dòng dữ liệu có thời gian không theo đúng chuẩn 30 phút
            initial_rows_before_time_filter = len(df_sanluong)

            # Lọc các dòng dữ liệu có phút là 0 hoặc 30 và giây là 0
            valid_time_mask = (
                (df_sanluong["TIME"].dt.minute == 0)
                | (df_sanluong["TIME"].dt.minute == 30)
            ) & (df_sanluong["TIME"].dt.second == 0)

            df_sanluong = df_sanluong[valid_time_mask]

            removed_time_rows = initial_rows_before_time_filter - len(df_sanluong)
            if removed_time_rows > 0:
                print(
                    f"   ⚠️ Đã loại bỏ {removed_time_rows:,} dòng có thời gian không theo chuẩn 30 phút."
                )

            # 2. Đảm bảo kiểu dữ liệu số cho CS và xử lý NaN
            df_sanluong["CS"] = pd.to_numeric(df_sanluong["CS"], errors="coerce")
            initial_rows = len(df_sanluong)
            df_sanluong.dropna(subset=["CS"], inplace=True)
            removed_rows = initial_rows - len(df_sanluong)
            if removed_rows > 0:
                print(
                    f"   ⚠️ Đã loại bỏ {removed_rows:,} dòng có giá trị CS không hợp lệ."
                )

            # 3. Tạo các cột thời gian cần thiết
            df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year.astype("int16")
            df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month.astype("int8")
            df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day.astype("int8")
            df_sanluong["TIME_SLOT"] = df_sanluong["TIME"].dt.strftime("%H:%M")
            df_sanluong["CTDL"] = df_sanluong["CTDL"].astype(str).str.strip()
            df_sanluong["NMTD"] = df_sanluong["NMTD"].astype(str).str.strip()

            # 4. Tối ưu kiểu dữ liệu Categorical cho các cột chuỗi lặp lại
            df_sanluong["CTDL"] = df_sanluong["CTDL"].astype("category")
            df_sanluong["NMTD"] = df_sanluong["NMTD"].astype("category")
            df_sanluong["MADIEMDO"] = df_sanluong["MADIEMDO"].astype("category")
            df_sanluong["TIME_SLOT"] = df_sanluong["TIME_SLOT"].astype("category")

            print("   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.")

            # 5. Tạo DataFrame được Index cho tính toán thống kê nhanh
            index_cols = ["CTDL", "NMTD", "MADIEMDO", "YEAR", "MONTH_NUM", "DAY_NUM"]
            # Chỉ cần TIME_SLOT và CS cho df_indexed
            df_sanluong_indexed = df_sanluong[index_cols + ["TIME_SLOT", "CS"]].copy()
            # Giữ kiểu category cho index để tiết kiệm bộ nhớ index
            df_sanluong_indexed.set_index(index_cols, inplace=True)
            df_sanluong_indexed.sort_index(
                inplace=True
            )  # Rất quan trọng cho hiệu năng loc
            print("   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.")

            print(
                f"⏱️ Thời gian tiền xử lý: {time.time() - start_process_time:.2f} giây"
            )

        except Exception as e:
            print(f"❌ Lỗi trong quá trình tiền xử lý: {e}")
            # Đảm bảo cả hai df đều rỗng nếu có lỗi
            df_sanluong = pd.DataFrame()
            df_sanluong_indexed = pd.DataFrame()
    else:
        print("ℹ️ Không có dữ liệu đầu vào để xử lý.")

    return df_sanluong, df_sanluong_indexed


def create_ui():
    """Tạo giao diện tương tác"""
    global df_sanluong, df_sanluong_indexed, ui_components, _observed_callbacks
    global nmtd_cache, madiemdo_cache

    if not df_sanluong.empty and not df_sanluong_indexed.empty:
        print("\n🔄 Tạo giao diện tương tác...")
        # ========== DANH SÁCH CHO DROPDOWN ==========
        try:
            ctdl_list = sorted(df_sanluong["CTDL"].cat.categories)
            year_list = sorted(df_sanluong["YEAR"].unique())
            nmtd_list_all = sorted(df_sanluong["NMTD"].cat.categories)
        except Exception as e:
            print(f"❌ Lỗi lấy danh sách dropdown: {e}")
            ctdl_list, year_list, nmtd_list_all = [], [], []

        # ========== TẠO DROPDOWNS ==========
        # Tab 1
        ctdl_dropdown_month_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_month_plot = make_dropdown([], "Nhà máy:", "300px")
        year_dropdown_month_plot = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_month_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
        placeholder_month_plot = widgets.Label(
            value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
        )
        # Tab 2
        ctdl_dropdown_day_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_day_plot = make_dropdown([], "Nhà máy:", "300px")
        year_dropdown_day_plot = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_day_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
        day_dropdown_day_plot = make_dropdown([], "Ngày:", "120px")
        # Tab 3
        ctdl_dropdown_stat = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_stat = make_dropdown([], "Nhà máy:", "300px")
        madiemdo_dropdown_stat = make_dropdown([], "Mã điểm đo:", "300px")
        year_dropdown_stat = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_stat = make_dropdown(list(range(1, 13)), "Tháng:", "120px")

        # ========== NHÓM WIDGET ==========
        month_plot_widgets = [
            ctdl_dropdown_month_plot,
            nmtd_dropdown_month_plot,
            year_dropdown_month_plot,
            month_dropdown_month_plot,
        ]
        day_plot_widgets = [
            ctdl_dropdown_day_plot,
            nmtd_dropdown_day_plot,
            year_dropdown_day_plot,
            month_dropdown_day_plot,
            day_dropdown_day_plot,
        ]
        stat_widgets = [
            ctdl_dropdown_stat,
            nmtd_dropdown_stat,
            madiemdo_dropdown_stat,
            year_dropdown_stat,
            month_dropdown_stat,
        ]

        # Lưu vào dictionary để tái sử dụng
        ui_components = {
            "month_plot_widgets": month_plot_widgets,
            "day_plot_widgets": day_plot_widgets,
            "stat_widgets": stat_widgets,
            "placeholder_month_plot": placeholder_month_plot,
            "ctdl_dropdown_month_plot": ctdl_dropdown_month_plot,
            "nmtd_dropdown_month_plot": nmtd_dropdown_month_plot,
            "year_dropdown_month_plot": year_dropdown_month_plot,
            "month_dropdown_month_plot": month_dropdown_month_plot,
            "ctdl_dropdown_day_plot": ctdl_dropdown_day_plot,
            "nmtd_dropdown_day_plot": nmtd_dropdown_day_plot,
            "year_dropdown_day_plot": year_dropdown_day_plot,
            "month_dropdown_day_plot": month_dropdown_day_plot,
            "day_dropdown_day_plot": day_dropdown_day_plot,
            "ctdl_dropdown_stat": ctdl_dropdown_stat,
            "nmtd_dropdown_stat": nmtd_dropdown_stat,
            "madiemdo_dropdown_stat": madiemdo_dropdown_stat,
            "year_dropdown_stat": year_dropdown_stat,
            "month_dropdown_stat": month_dropdown_stat,
        }

        # ========== GẮN OBSERVE BAN ĐẦU ==========
        ctdl_dropdown_month_plot.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_month_plot),
            names="value",
        )
        ctdl_dropdown_day_plot.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_day_plot),
            names="value",
        )
        ctdl_dropdown_stat.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_stat), names="value"
        )
        nmtd_dropdown_stat.observe(
            lambda change: update_madiemdo(
                ctdl_dropdown_stat.value, change["new"], madiemdo_dropdown_stat
            ),
            names="value",
        )

        # Khởi tạo
        if ctdl_dropdown_month_plot.value:
            update_nmtd(ctdl_dropdown_month_plot.value, nmtd_dropdown_month_plot)
        if ctdl_dropdown_day_plot.value:
            update_nmtd(ctdl_dropdown_day_plot.value, nmtd_dropdown_day_plot)
        if ctdl_dropdown_stat.value:
            update_nmtd(ctdl_dropdown_stat.value, nmtd_dropdown_stat)
        if ctdl_dropdown_stat.value and nmtd_dropdown_stat.value:
            update_madiemdo(
                ctdl_dropdown_stat.value,
                nmtd_dropdown_stat.value,
                madiemdo_dropdown_stat,
            )

        year_dropdown_day_plot.observe(update_day_options, names="value")
        month_dropdown_day_plot.observe(update_day_options, names="value")
        if year_dropdown_day_plot.value and month_dropdown_day_plot.value:
            update_day_options()

        # ========== NHÓM OBSERVER ==========
        for w in month_plot_widgets:
            _observed_callbacks[w] = {"func": on_change_month_plot, "active": False}
        for w in day_plot_widgets:
            _observed_callbacks[w] = {"func": on_change_day_plot, "active": False}
        for w in stat_widgets:
            _observed_callbacks[w] = {"func": on_change_stat, "active": False}

        # ========== TẠO GIAO DIỆN TABS ==========
        tab_month_plot = widgets.HBox(
            month_plot_widgets + [placeholder_month_plot],
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tab_day_plot = widgets.HBox(
            day_plot_widgets,
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tab_stat = widgets.HBox(
            stat_widgets,
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tabs = widgets.Tab(children=[tab_month_plot, tab_day_plot, tab_stat])
        tabs.set_title(0, "📅 Biểu đồ tháng")
        tabs.set_title(1, "🗓️ Biểu đồ ngày")
        tabs.set_title(2, "📊 Thống kê tháng")

        out = widgets.Output()
        ui_components["tabs"] = tabs
        ui_components["out"] = out
        ui_components["tab_month_plot"] = tab_month_plot
        ui_components["tab_day_plot"] = tab_day_plot
        ui_components["tab_stat"] = tab_stat

        # Gắn xử lý chuyển tab
        tabs.observe(on_tab_change, names="selected_index")

        return ui_components
    else:
        print("\n❌ Dữ liệu không khả dụng hoặc rỗng.")
        return None


def show_ui():
    """Hiển thị giao diện tương tác"""
    global ui_components

    if "tabs" in ui_components and "out" in ui_components:
        print("✅ Giao diện sẵn sàng!")
        display(ui_components["tabs"], ui_components["out"])

        # Kích hoạt lần đầu
        on_tab_change(
            {
                "name": "selected_index",
                "old": None,
                "new": ui_components["tabs"].selected_index,
                "owner": ui_components["tabs"],
                "type": "change",
            }
        )
    else:
        print("❌ Giao diện chưa được khởi tạo.")


# ===== HÀM HỖ TRỢ =====
def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
    """Tạo dropdown widget"""
    safe_options = list(options) if options is not None else []
    return widgets.Dropdown(
        options=safe_options,
        description=description,
        value=safe_options[0] if safe_options else None,
        layout=widgets.Layout(width=width, margin=margin),
        style={"description_width": "auto"},
        disabled=not bool(safe_options),
    )


def update_nmtd(ctdl_value, dropdown_to_update):
    """Cập nhật danh sách nhà máy theo CTDL"""
    global nmtd_cache, df_sanluong_indexed

    options = []
    current_nmtd_value = dropdown_to_update.value

    if ctdl_value:
        if ctdl_value in nmtd_cache:  # Kiểm tra cache trước
            options = nmtd_cache[ctdl_value]
        elif not df_sanluong_indexed.empty:
            try:
                # Lấy NMTD categories từ index con tương ứng với CTDL
                nmtd_options = (
                    df_sanluong_indexed.loc[ctdl_value]
                    .index.get_level_values("NMTD")
                    .unique()
                    .tolist()
                )
                options = sorted(nmtd_options)
                nmtd_cache[ctdl_value] = options  # Lưu vào cache
            except KeyError:
                options = []  # CTDL không có trong index
            except Exception as e:
                print(f"Lỗi cập nhật NMTD cache cho '{ctdl_value}': {e}")
                options = []

    # Cập nhật dropdown
    dropdown_to_update.options = options
    dropdown_to_update.disabled = not bool(options)
    if current_nmtd_value in options:
        dropdown_to_update.value = current_nmtd_value
    elif options:
        dropdown_to_update.value = options[0]
    else:
        dropdown_to_update.value = None


def update_madiemdo(ctdl_value, nmtd_value, dropdown_to_update):
    """Cập nhật danh sách mã điểm đo theo CTDL và NMTD"""
    global madiemdo_cache, df_sanluong

    options = []
    current_madiemdo_value = dropdown_to_update.value

    if ctdl_value and nmtd_value:
        cache_key = f"{ctdl_value}_{nmtd_value}"
        if cache_key in madiemdo_cache:
            options = madiemdo_cache[cache_key]
        elif not df_sanluong.empty:
            try:
                # Lấy danh sách các mã điểm đo của nhà máy đã chọn
                madiemdo_options = (
                    df_sanluong[
                        (df_sanluong["CTDL"] == ctdl_value)
                        & (df_sanluong["NMTD"] == nmtd_value)
                    ]["MADIEMDO"]
                    .unique()
                    .tolist()
                )
                options = sorted(madiemdo_options)
                madiemdo_cache[cache_key] = options  # Lưu vào cache
            except Exception as e:
                print(f"Lỗi cập nhật MADIEMDO cache cho '{cache_key}': {e}")
                options = []

    # Cập nhật dropdown
    dropdown_to_update.options = options
    dropdown_to_update.disabled = not bool(options)
    if current_madiemdo_value in options:
        dropdown_to_update.value = current_madiemdo_value
    elif options:
        dropdown_to_update.value = options[0]
    else:
        dropdown_to_update.value = None


def update_day_options(*args):
    """Cập nhật danh sách ngày theo năm và tháng"""
    global ui_components

    year = ui_components["year_dropdown_day_plot"].value
    month = ui_components["month_dropdown_day_plot"].value
    current_day = ui_components["day_dropdown_day_plot"].value
    day_options = []

    if year and month:
        try:
            max_day = calendar.monthrange(year, month)[1]
            day_options = list(range(1, max_day + 1))
        except Exception as e:
            print(f"Lỗi lấy số ngày {month}/{year}: {e}")

    ui_components["day_dropdown_day_plot"].options = day_options
    ui_components["day_dropdown_day_plot"].disabled = not bool(day_options)

    if current_day in day_options:
        ui_components["day_dropdown_day_plot"].value = current_day
    elif day_options:
        ui_components["day_dropdown_day_plot"].value = 1
    else:
        ui_components["day_dropdown_day_plot"].value = None


def on_change_month_plot(change):
    """Xử lý sự kiện thay đổi dropdown trong tab biểu đồ tháng"""
    global ui_components

    if ui_components["tabs"].selected_index == 0 and all(
        w.value is not None for w in ui_components["month_plot_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_filtered(
                "month",
                ui_components["ctdl_dropdown_month_plot"].value,
                ui_components["nmtd_dropdown_month_plot"].value,
                ui_components["year_dropdown_month_plot"].value,
                ui_components["month_dropdown_month_plot"].value,
            )


def on_change_day_plot(change):
    """Xử lý sự kiện thay đổi dropdown trong tab biểu đồ ngày"""
    global ui_components

    if ui_components["tabs"].selected_index == 1 and all(
        w.value is not None for w in ui_components["day_plot_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_filtered(
                "day",
                ui_components["ctdl_dropdown_day_plot"].value,
                ui_components["nmtd_dropdown_day_plot"].value,
                ui_components["year_dropdown_day_plot"].value,
                ui_components["month_dropdown_day_plot"].value,
                ui_components["day_dropdown_day_plot"].value,
            )


def on_change_stat(change):
    """Xử lý sự kiện thay đổi dropdown trong tab thống kê"""
    global ui_components

    if ui_components["tabs"].selected_index == 2 and all(
        w.value is not None for w in ui_components["stat_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_timeslot_stats(
                ui_components["ctdl_dropdown_stat"].value,
                ui_components["nmtd_dropdown_stat"].value,
                ui_components["madiemdo_dropdown_stat"].value,
                ui_components["year_dropdown_stat"].value,
                ui_components["month_dropdown_stat"].value,
            )


def disable_observe(widgets_list):
    """Tắt observer cho các widget"""
    global _observed_callbacks

    for w in widgets_list:
        if w in _observed_callbacks and _observed_callbacks[w]["active"]:
            try:
                w.unobserve(_observed_callbacks[w]["func"], names="value")
                _observed_callbacks[w]["active"] = False
            except:
                _observed_callbacks[w]["active"] = False


def enable_observe(widgets_list):
    """Bật observer cho các widget"""
    global _observed_callbacks

    for w in widgets_list:
        if (
            w in _observed_callbacks
            and not w.disabled
            and not _observed_callbacks[w]["active"]
        ):
            try:
                w.observe(_observed_callbacks[w]["func"], names="value")
                _observed_callbacks[w]["active"] = True
            except Exception as e:
                print(f"Lỗi observe {w.description}: {e}")


def on_tab_change(change):
    """Xử lý sự kiện chuyển tab"""
    global ui_components

    if change["name"] == "selected_index":
        new_tab_index = change["new"]
        old_tab_index = change["old"]

        # --- Tắt observer tab cũ ---
        if old_tab_index == 0:
            disable_observe(ui_components["month_plot_widgets"])
        elif old_tab_index == 1:
            disable_observe(ui_components["day_plot_widgets"])
        elif old_tab_index == 2:
            disable_observe(ui_components["stat_widgets"])

        # --- Xác định nguồn đồng bộ ---
        source_widgets = {}
        if old_tab_index == 0:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }
        elif old_tab_index == 1:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_day_plot"],
                "nmtd": ui_components["nmtd_dropdown_day_plot"],
                "year": ui_components["year_dropdown_day_plot"],
                "month": ui_components["month_dropdown_day_plot"],
                "day": ui_components["day_dropdown_day_plot"],
            }
        elif old_tab_index == 2:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_stat"],
                "nmtd": ui_components["nmtd_dropdown_stat"],
                "year": ui_components["year_dropdown_stat"],
                "month": ui_components["month_dropdown_stat"],
            }
        else:  # Khởi tạo
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }

        # --- Xác định widgets đích của tab mới ---
        target_widgets = {}
        if new_tab_index == 0:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }
        elif new_tab_index == 1:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_day_plot"],
                "nmtd": ui_components["nmtd_dropdown_day_plot"],
                "year": ui_components["year_dropdown_day_plot"],
                "month": ui_components["month_dropdown_day_plot"],
                "day": ui_components["day_dropdown_day_plot"],
            }
        elif new_tab_index == 2:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_stat"],
                "nmtd": ui_components["nmtd_dropdown_stat"],
                "madiemdo": ui_components["madiemdo_dropdown_stat"],
                "year": ui_components["year_dropdown_stat"],
                "month": ui_components["month_dropdown_stat"],
            }

        # --- Đồng bộ từ nguồn sang đích ---
        if old_tab_index != new_tab_index and old_tab_index is not None:
            # Đồng bộ CTDL
            if source_widgets.get("ctdl") and source_widgets["ctdl"].value is not None:
                target_widgets["ctdl"].value = source_widgets["ctdl"].value

            # Đồng bộ NMTD (phải cập nhật options trước)
            target_nmtd = target_widgets["nmtd"]
            update_nmtd(target_widgets["ctdl"].value, target_nmtd)
            if (
                source_widgets.get("nmtd")
                and source_widgets["nmtd"].value in target_nmtd.options
            ):
                target_nmtd.value = source_widgets["nmtd"].value

            # Đồng bộ MADIEMDO (phải cập nhật options trước)
            if new_tab_index == 2 and "madiemdo" in target_widgets:
                update_madiemdo(
                    target_widgets["ctdl"].value,
                    target_widgets["nmtd"].value,
                    target_widgets["madiemdo"],
                )

            # Đồng bộ Year
            if source_widgets.get("year") and source_widgets["year"].value is not None:
                target_widgets["year"].value = source_widgets["year"].value

            # Đồng bộ Month
            if (
                source_widgets.get("month")
                and source_widgets["month"].value is not None
            ):
                target_widgets["month"].value = source_widgets["month"].value

            # Nếu chuyển sang tab ngày, cập nhật dropdown ngày
            if "day" in target_widgets:
                update_day_options()  # Cập nhật options ngày dựa trên năm/tháng đã chọn
                # Mặc định chọn ngày 1 nếu đến từ tab khác
                if old_tab_index != 1 and target_widgets["day"].options:
                    target_widgets["day"].value = 1

        # --- Kích hoạt observer và cập nhật nội dung tab mới ---
        if new_tab_index == 0:
            enable_observe(ui_components["month_plot_widgets"])
            on_change_month_plot(None)
        elif new_tab_index == 1:
            enable_observe(ui_components["day_plot_widgets"])
            on_change_day_plot(None)
        elif new_tab_index == 2:
            enable_observe(ui_components["stat_widgets"])
            on_change_stat(None)


def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
    """Vẽ biểu đồ theo các tham số lọc"""
    global df_sanluong, ui_components

    with ui_components["out"]:
        clear_output(wait=True)
        plot_start_time = time.time()

        # Kiểm tra đầu vào
        required_vals = [ctdl, nmtd, year, month]
        if mode == "day":
            required_vals.append(day)
        if not all(v is not None for v in required_vals):
            print("   ⚠️ Vui lòng chọn đủ các bộ lọc.")
            return

        filtered_df = pd.DataFrame()
        try:
            # Tạo mask boolean hiệu quả
            mask = (
                (df_sanluong["CTDL"] == ctdl)
                & (df_sanluong["NMTD"] == nmtd)
                & (df_sanluong["YEAR"] == year)
                & (df_sanluong["MONTH_NUM"] == month)
            )
            if mode == "day":
                mask &= df_sanluong["DAY_NUM"] == day

            # Chỉ lấy các cột cần thiết cho vẽ
            filtered_df = df_sanluong.loc[mask, ["TIME", "CS", "MADIEMDO"]].copy()
            # Sắp xếp theo TIME
            filtered_df.sort_values("TIME", inplace=True)

        except Exception as e:
            print(f"❌ Lỗi khi lọc dữ liệu cho biểu đồ: {e}")

        if filtered_df.empty:
            print("   ⚠️ Không có dữ liệu cho lựa chọn này!")
            return

        # Vẽ biểu đồ
        try:
            title_ext = (
                f"{month}/{year}" if mode == "month" else f"{day}/{month}/{year}"
            )
            fig = px.line(
                filtered_df,
                x="TIME",
                y="CS",
                color="MADIEMDO",
                render_mode="webgl",
                markers=True if mode == "day" else False,
            )

            # Áp dụng layout
            fig.update_layout(
                title=None,
                annotations=[
                    dict(
                        text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                        xref="paper",
                        yref="paper",
                        x=0.45,
                        y=1.1,
                        xanchor="center",
                        yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black"),
                    )
                ],
                template="plotly_white",
                height=600,
                hovermode="x unified",
                xaxis_title=None,
                yaxis_title="Giá trị công suất",
                xaxis=dict(
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                yaxis=dict(
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    rangemode="tozero",
                    title_font=dict(size=14, color="black", weight="bold"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                margin=dict(l=60, r=40, t=80, b=80),
                paper_bgcolor="white",
                plot_bgcolor="#f0f8ff",
            )
            fig.add_annotation(
                text="<b>Thời gian</b>",
                xref="paper",
                yref="paper",
                x=0.45,
                y=-0.15,
                showarrow=False,
                font=dict(size=14, color="black"),
            )
            fig.update_traces(
                line=dict(width=2),
                hovertemplate="Công suất: %{y}",
            )

            fig.show()

        except Exception as e:
            print(f"❌ Lỗi khi vẽ biểu đồ Plotly Express: {e}")


def plot_timeslot_stats(ctdl, nmtd, madiemdo, year, month):
    """Vẽ biểu đồ thống kê theo khung giờ"""
    global df_sanluong_indexed, ui_components

    with ui_components["out"]:
        clear_output(wait=True)

        if not all([ctdl, nmtd, year, month]):
            print("   ⚠️ Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
            return

        stats_by_time = pd.DataFrame()
        try:
            # Dùng .loc trên df_sanluong_indexed
            idx_query = (ctdl, nmtd, madiemdo, year, month)
            # Kiểm tra sự tồn tại của index
            if idx_query in df_sanluong_indexed.index.droplevel("DAY_NUM").unique():
                monthly_data = df_sanluong_indexed.loc[idx_query, ["TIME_SLOT", "CS"]]

                if not monthly_data.empty:
                    # Group và Agg
                    stats_by_time = monthly_data.groupby("TIME_SLOT", observed=True)[
                        "CS"
                    ].agg(
                        min="min",
                        p25=lambda x: x.quantile(0.25),
                        p50="mean",
                        p75=lambda x: x.quantile(0.75),
                        max="max",
                    )
                    # Sắp xếp index
                    stats_by_time = stats_by_time.sort_index()
                else:
                    print("   ℹ️ Không có dữ liệu chi tiết sau khi lọc.")

        except KeyError:
            print("   ℹ️ Không tìm thấy dữ liệu (KeyError).")
        except Exception as e:
            print(f"❌ Lỗi tính toán thống kê: {e}")

        if stats_by_time.empty:
            print("   ℹ️ Không có dữ liệu cho lựa chọn này!")
            return

        # Vẽ biểu đồ
        try:
            fig = go.Figure()
            time_slots = stats_by_time.index
            p25 = stats_by_time["p25"]
            p75 = stats_by_time["p75"]
            p50 = stats_by_time["p50"]
            min_cs = stats_by_time["min"]
            max_cs = stats_by_time["max"]

            # 1. Vùng Min-Max
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=max_cs,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                    hoverinfo="skip",
                    hovertemplate="Max: %{y:.2f}<extra></extra>",
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=min_cs,
                    mode="lines",
                    line=dict(width=0),
                    fill="tonexty",
                    fillcolor="rgba(211, 211, 211, 0.5)",
                    name="Min-Max",
                    hoverinfo="skip",
                    hovertemplate="Min: %{y:.2f}<extra></extra>",
                )
            )

            # 2. Vùng P25-P75
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p75,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                    hoverinfo="skip",
                    hovertemplate="P75: %{y:.2f}<extra></extra>",
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p25,
                    mode="lines",
                    line=dict(width=0),
                    fill="tonexty",
                    fillcolor="rgba(173, 216, 230, 0.6)",
                    name="Tần suất 25-75",
                    hoverinfo="skip",
                    hovertemplate="P25: %{y:.2f}<extra></extra>",
                )
            )

            # 3. Đường Median (P50)
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p50,
                    mode="lines+markers",
                    marker=dict(size=6, color="blue"),
                    line=dict(color="blue", width=2.5),
                    name="Trung bình",
                    hovertemplate="Mean: %{y:.2f}<extra></extra>",
                )
            )

            # Layout
            fig.update_layout(
                title=None,
                annotations=[
                    dict(
                        text=f"<b>Thống kê công suất theo khung giờ - {nmtd} - {month}/{year}</b>",
                        xref="paper",
                        yref="paper",
                        x=0.45,
                        y=1.1,
                        xanchor="center",
                        yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black"),
                    )
                ],
                template="plotly_white",
                height=600,
                hovermode="x unified",
                xaxis_title=None,
                yaxis_title="Giá trị công suất",
                xaxis=dict(
                    tickmode="array",
                    tickvals=time_slots,
                    ticktext=[t for t in time_slots],
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                    tickangle=-90,
                ),
                yaxis=dict(
                    rangemode="tozero",
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black", weight="bold"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
                margin=dict(l=60, r=40, t=80, b=80),
                paper_bgcolor="white",
                plot_bgcolor="#f0f8ff",
            )
            fig.add_annotation(
                text="<b>Khung giờ</b>",
                xref="paper",
                yref="paper",
                x=0.45,
                y=-0.18,
                showarrow=False,
                font=dict(size=14, color="black"),
            )
            fig.show()

        except Exception as e:
            print(f"❌ Lỗi vẽ biểu đồ thống kê Plotly GO: {e}")


# Hàm chạy toàn bộ quy trình một lần
def run_full_process(parquet_folder=parquet_folder):
    """Chạy toàn bộ quy trình đọc dữ liệu, xử lý và hiển thị giao diện"""
    read_data(parquet_folder)
    preprocess_data()
    create_ui()
    show_ui()


# Hàm tùy chỉnh chỉ hiển thị UI (nếu dữ liệu đã được xử lý)
def reload_ui():
    """Tải lại UI từ dữ liệu đã xử lý"""
    global df_sanluong, df_sanluong_indexed

    if not df_sanluong.empty and not df_sanluong_indexed.empty:
        create_ui()
        show_ui()
    else:
        print("❌ Cần phải đọc và xử lý dữ liệu trước.")


# Khởi chạy quy trình khi cell được chạy
if __name__ == "__main__":
    # Nếu chạy lần đầu tiên
    if df_all.empty:
        run_full_process()
    # Nếu đã có dữ liệu và chỉ cần tải lại UI
    elif not df_sanluong.empty and not df_sanluong_indexed.empty:
        reload_ui()
    else:
        print("⚠️ Dữ liệu đã được đọc nhưng chưa xử lý.")
        preprocess_data()
        create_ui()
        show_ui()

🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 29.48 giây

🔄 Bắt đầu tiền xử lý và tối ưu hóa...
   ⚠️ Đã loại bỏ 60,706 dòng có thời gian không theo chuẩn 30 phút.
   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.
   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.
⏱️ Thời gian tiền xử lý: 143.82 giây

🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()

In [8]:
# Tôi muốn lấy dữ liệu từ NMTĐ Nâm Hóa 
# First, filter for NMTĐ NẬM HÓA
df_namhoa = df_all[df_all["NMTD"] == "NMTĐ NẬM HÓA"]

# Then filter for the date range (April 19, 2024 to July 9, 2024)
start_date = '2024-04-19'
end_date = '2024-07-09'
df_namhoa = df_namhoa[(df_namhoa["ENDTIME"] >= start_date) & (df_namhoa["ENDTIME"] <= end_date)]
# sắp xếp lại dữ liệu theo thứ tự thời gian
df_namhoa = df_namhoa.sort_values(by="ENDTIME")
df_namhoa

,CTDL,NMTD,MADIEMDO,ENDTIME,CS
15170566,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 00:00:00,0.0
15170549,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 00:30:00,0.0
15170554,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 01:00:00,0.0
15170548,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 01:30:00,0.0
15170553,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 02:00:00,0.0
...,...,...,...,...,...
15174359,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 22:00:00,3151.4
15174355,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 22:30:00,3116.3
15174358,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 23:00:00,3091.2
15174481,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 23:30:00,3071.1


In [9]:
# đọc dữ liệu từ file parquet
df_DGMS = pd.read_parquet(r"C:\Khue\TDN\data\interim\BCSVH_Bac_2024.parquet")
df_DGMS

,CTDL,NMTD,Q.ĐK,P_rated,ENDTIME,CS
0,PC_BACKAN,Nặm Cắt,PC_BACKAN,3.2,2024-04-19 00:30:00,0.0
1,PC_BACKAN,Pác Cáp,PC_BACKAN,6.0,2024-04-19 00:30:00,0.0
2,PC_BACKAN,Tà Làng,PC_BACKAN,4.5,2024-04-19 00:30:00,0.0
3,PC_BACKAN,Thác Giềng,PC_BACKAN,5.5,2024-04-19 00:30:00,0.0
4,PC_BACKAN,Thượng Ân,PC_BACKAN,2.4,2024-04-19 00:30:00,0.0
...,...,...,...,...,...,...
3918763,PC_YENBAI,Nậm Tục,PC_YENBAI,3.0,2025-01-01 00:00:00,0.0
3918764,PC_YENBAI,Ngòi Hút 1,PC_YENBAI,8.4,2025-01-01 00:00:00,0.0
3918765,PC_YENBAI,Phình Hồ,PC_YENBAI,2.5,2025-01-01 00:00:00,0.0
3918766,PC_YENBAI,Sài Lương,PC_YENBAI,4.5,2025-01-01 00:00:00,0.0


In [10]:
df_namhoa_1 = df_DGMS[df_DGMS["NMTD"] == "Nậm Hóa 2"]

# Then filter for the date range (April 19, 2024 to July 9, 2024)
start_date = '2024-04-19'
end_date = '2024-07-09'
df_namhoa_1 = df_namhoa_1[(df_namhoa_1["ENDTIME"] >= start_date) & (df_namhoa_1["ENDTIME"] <= end_date)]
# sắp xếp lại dữ liệu theo thứ tự thời gian
df_namhoa_1 = df_namhoa_1.sort_values(by="ENDTIME")
df_namhoa_1

,CTDL,NMTD,Q.ĐK,P_rated,ENDTIME,CS
2800308,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-04-19 00:00:00,0.00
2800340,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-04-19 00:30:00,0.00
2800394,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-04-19 01:00:00,0.00
2800448,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-04-19 01:30:00,0.00
2800502,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-04-19 02:00:00,0.00
...,...,...,...,...,...,...
3010022,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-07-08 22:00:00,1.59
3010076,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-07-08 22:30:00,1.57
3010130,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-07-08 23:00:00,1.56
3010184,PC_SONLA,Nậm Hóa 2,A1,8.0,2024-07-08 23:30:00,1.55


In [20]:
# Tôi muốn tính chênh lệch giá trị CS giữa df_namhoa và df_namhoa_1
# Đảm bảo rằng cả hai DataFrame đều đã được sắp xếp theo ENDTIME
# Merge the two DataFrames on ENDTIME only
merged_df = pd.merge(df_namhoa, df_namhoa_1, on=["ENDTIME"], suffixes=('_namhoa', '_namhoa_1'))
# Tính toán chênh lệch giá trị CS
merged_df['CS_diff'] = merged_df['CS_namhoa']/1000 - merged_df['CS_namhoa_1']
# Hiển thị DataFrame kết quả
merged_df

,CTDL_namhoa,NMTD_namhoa,MADIEMDO,ENDTIME,CS_namhoa,CTDL_namhoa_1,NMTD_namhoa_1,Q.ĐK,P_rated,CS_namhoa_1,CS_diff
0,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 00:00:00,0.0,PC_SONLA,Nậm Hóa 2,A1,8.0,0.00,0.0000
1,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 00:30:00,0.0,PC_SONLA,Nậm Hóa 2,A1,8.0,0.00,0.0000
2,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 01:00:00,0.0,PC_SONLA,Nậm Hóa 2,A1,8.0,0.00,0.0000
3,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 01:30:00,0.0,PC_SONLA,Nậm Hóa 2,A1,8.0,0.00,0.0000
4,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-04-19 02:00:00,0.0,PC_SONLA,Nậm Hóa 2,A1,8.0,0.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
3885,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 22:00:00,3151.4,PC_SONLA,Nậm Hóa 2,A1,8.0,1.59,1.5614
3886,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 22:30:00,3116.3,PC_SONLA,Nậm Hóa 2,A1,8.0,1.57,1.5463
3887,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 23:00:00,3091.2,PC_SONLA,Nậm Hóa 2,A1,8.0,1.56,1.5312
3888,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ NẬM HÓA,G2A035S000M171,2024-07-08 23:30:00,3071.1,PC_SONLA,Nậm Hóa 2,A1,8.0,1.55,1.5211


In [21]:
# # Bỏ những giá trị bằng 0 của CS_diff
# merged_df = merged_df[merged_df['CS_diff'] != 0]
# Tính trị tuyệt đối của CS_diff
merged_df['CS_diff'] = merged_df['CS_diff'].abs()
# Tính giá trị trung bình của CS_diff
average_diff = merged_df['CS_diff'].mean()
average_diff


1.331453589333162